In [1]:
import pandas as pd
import numpy as np
import re

# Read CSV file into a dataframe
torino_houses = pd.read_csv('houses-torino-detailed-df.csv')

# Show full column width
pd.set_option('display.max_colwidth', None)

# Checking what the data looks like

In [2]:
torino_houses.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL
0,"Apartment via Albenga 2, Lingotto, Turin","€ 165,000",5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100 m²,NaN,"3°, with lift",Apartment | Full ownership | Medium property class,9 floors,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin","€ 71,000",2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60 m²,NaN,"6°, with lift",Apartment | Full ownership | Medium property class,6 floors,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/
2,"Apartment corso siracusa, 79, Santa Rita, Turin","€ 235,000",5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128 m² | commercial 134.5 m² - See detail,NaN,"2°, with lift",Apartment | Full ownership | Stately property class,7 floors,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin","€ 735,000",5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235 m²,3,"4°, with lift",Apartment | Full ownership | Stately property class,4 floors,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin","€ 248,000",4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145 m²,2,"3°, with lift",Apartment | Full ownership | Stately property class,7 floors,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/


In [3]:
torino_houses.describe()

,Year of construction
count,1877.000000
mean,1946.795951
std,67.721306
min,1065.000000
25%,1940.000000
50%,1960.000000
75%,1970.000000
max,2024.000000


# Checking for missing values

In [4]:
torino_houses.isnull().sum()

Address description         2
Price                      14
Rooms                      54
Rooms detailed              0
Surface area               14
Bathrooms                1322
Floor                      22
House type                  2
Total building floors      39
Year of construction      123
Condition                   0
Energy Efficiency         406
URL                         0
dtype: int64

There are quite some missing values. From opening the file in a CSV reader, I can see there are 2 rows with almost all data points missing. Is there a way to do this in Pandas? I.e., to count how many columns are missing in each row?

In [5]:
torino_houses.shape

(2000, 13)

In [6]:
# This uses a threshold of at least 4 columns missing for the row to be dropped
# th = torino_houses.dropna(axis=0, thresh=4)

# This uses the column 'Adress description' to see which NAs to drop. It drops the rows with nas in 'Address description'
th = torino_houses.dropna(axis=0, subset='Address description')

In [7]:
th.shape

(1998, 13)

# Check for duplicates

In [8]:
th.duplicated().sum()

0

There don't seem to be any duplicates.

In [9]:
th.duplicated(subset=['Address description', 'URL']).sum()

0

There don't seem to be any duplicates in the address description or URL columns.

# Cleaning string data, splitting it, filling missing data with descriptive string other columns

In [10]:
# Split the 'Rooms detailed' column so that the total number of rooms is in a separate column
# Later, we can use the total in case it is missing from the other Rooms column
th[['Rooms detailed total', 'Other room details']] = th['Rooms detailed'].str.split('(', expand=True)

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3737026628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th[['Rooms detailed total', 'Other room details']] = th['Rooms detailed'].str.split('(', expand=True)
C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3737026628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th[['Rooms detailed total', 'Other room details']] = th['Rooms detailed'].str.split('(', expand=True)


In [11]:
th.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details
0,"Apartment via Albenga 2, Lingotto, Turin","€ 165,000",5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100 m²,NaN,"3°, with lift",Apartment | Full ownership | Medium property class,9 floors,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,"3 bedrooms, 2 others), 1 bathroom, kitchen diner"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin","€ 71,000",2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60 m²,NaN,"6°, with lift",Apartment | Full ownership | Medium property class,6 floors,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,"1 bedroom, 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin","€ 235,000",5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128 m² | commercial 134.5 m² - See detail,NaN,"2°, with lift",Apartment | Full ownership | Stately property class,7 floors,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,"4 bedrooms, 1 other), 1 bathroom, kitchen diner"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin","€ 735,000",5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235 m²,3,"4°, with lift",Apartment | Full ownership | Stately property class,4 floors,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,"2 bedrooms, 5 others), 3 bathrooms, kitchen diner"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin","€ 248,000",4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145 m²,2,"3°, with lift",Apartment | Full ownership | Stately property class,7 floors,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,"2 bedrooms, 2 others), 2 bathrooms, kitchen diner"


In [12]:
# Remove 'floors' text from Total building floors column
th['Total building floors'] = th['Total building floors'].str.replace(' floors', '')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\977619179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Total building floors'] = th['Total building floors'].str.replace(' floors', '')


In [13]:
# Remove whitespace from Total building floors column
th['Total building floors'] = th['Total building floors'].str.strip()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\2018952879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Total building floors'] = th['Total building floors'].str.strip()


In [14]:
th['Total building floors']

0       9
1       6
2       7
3       4
4       7
       ..
1995    5
1996    5
1997    4
1998    5
1999    4
Name: Total building floors, Length: 1998, dtype: object

In [15]:
# Checking how many rows in the Total building floors column
remaining_floor_text = th['Total building floors'].str.contains(pat='[a-z]', case=False).sum()

print(remaining_floor_text)

65


There are 65 rows left with text. In theory there should still be a few with 'floor'.

In [16]:
# Check the unique values in 'Total building floors'
th['Total building floors'].unique()

array(['9', '6', '7', '4', '5', '2', '8', '10', '3', '1 floor', '15', nan,
       '16', '11', '14'], dtype=object)

Indeed, it seems that we still have some rows with '1 floor'. Let's check how many we have:

In [17]:
# Check how many rows in the 'Total building floors' column have the value of '1 floor'
th[th['Total building floors'] == '1 floor']['Total building floors'].count()

65

Great, so all of the rows with text are the same, '1 floor'. Let's remove the text:

In [18]:
# Remove, i.e., replace the ' floor' text with an empty string
th['Total building floors'] = th['Total building floors'].str.replace(' floor', '')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\2569796542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Total building floors'] = th['Total building floors'].str.replace(' floor', '')


Checking again, we don't have any rows left with text:

In [19]:
# Check how many rows in the 'Total building floors' column have the value of '1 floor'
th[th['Total building floors'] == '1 floor']['Total building floors'].count()

0

In [20]:
# Checking how many rows in the Total building floors column
remaining_floor_text = th['Total building floors'].str.contains(pat='[a-z]', case=False).sum()

print(remaining_floor_text)

0


Finally, let's just check how many NaNs there are in this column:

In [21]:
th['Total building floors'].isnull().sum()

37

We'll impute this column later, or check other columns to see whether this info is available elsewhere. For now, this column is clean and we can convert it to a numerical column:

In [22]:
th['Total building floors'] = th['Total building floors'].astype(float)

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\2600849856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Total building floors'] = th['Total building floors'].astype(float)


Let's continue cleaning other columns, like price:

In [23]:
# Remove euros sign from price column
th['Price'] = th['Price'].str.replace('€ ', '')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1047288970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Price'] = th['Price'].str.replace('€ ', '')


In [24]:
th['Price'].head()

0    165,000
1     71,000
2    235,000
3    735,000
4    248,000
Name: Price, dtype: object

Looking at the CSV file with a CSV reader, it seems that there are some values in the Price column that are text - 'price on application'. Since price is the target variable, let's drop these rows. First checking how many there are exactly:

In [25]:
# Checking how many rows in the Price column have text
price_text_rows = th['Price'].str.contains(pat='[a-z]', case=False).sum()

print(price_text_rows)

19


And checking how many of those 19 are the text 'price on application':

In [26]:
# Checking how many rows in the Price column have the text 'price on application'
price_text_poa = th['Price'].str.contains(pat='price on application', case=False).sum()

print(price_text_poa)

19


Okay, because our target variable is Price, and we have 19 rows with missing prices, we will drop these 19 rows. Maybe later we can check if we can impute them somehow, but I feel this would create bias in the model.

In [27]:
# Drop rows with value of 'price on application' in Price column
th_no_missing_price = th.drop(th[th['Price'] == 'price on application'].index)

In [28]:
# Check that 'price on application' rows are dropped
price_text_poa = th_no_missing_price['Price'].str.contains(pat='price on application', case=False).sum()

print(price_text_poa)

0


In [29]:
# We still have commas in our Price column. Let's remove them too
th_no_missing_price['Price'] = th_no_missing_price['Price'].str.replace(',', '')

th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details
0,"Apartment via Albenga 2, Lingotto, Turin",165000,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100 m²,NaN,"3°, with lift",Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,"3 bedrooms, 2 others), 1 bathroom, kitchen diner"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60 m²,NaN,"6°, with lift",Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,"1 bedroom, 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128 m² | commercial 134.5 m² - See detail,NaN,"2°, with lift",Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,"4 bedrooms, 1 other), 1 bathroom, kitchen diner"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235 m²,3,"4°, with lift",Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,"2 bedrooms, 5 others), 3 bathrooms, kitchen diner"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145 m²,2,"3°, with lift",Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,"2 bedrooms, 2 others), 2 bathrooms, kitchen diner"


In [30]:
th_no_missing_price.shape

(1979, 15)

In [31]:
# FInally, check how many Price rows are missing
print(th_no_missing_price['Price'].isnull().sum())

12


In [32]:
th_no_missing_price = th_no_missing_price.dropna(subset=['Price'])

In [33]:
# There should not be missing values in Price anymore
print(th_no_missing_price['Price'].isnull().sum())

0


In [34]:
th_no_missing_price.shape

(1967, 15)

In [35]:
# Just to be sure, let's strip whitespace from the price column
th_no_missing_price["Price"] = th_no_missing_price['Price'].str.strip()

Okay, we have cleaned the price column and removed those rows that had 'price on application' text under Price. We can now finalize this column by also converting it to a numerical column:

In [36]:
th_no_missing_price["Price"] = th_no_missing_price["Price"].astype(float)

Next, let's clean up surface area. We can start by checking if there are any missing values. Then, we should try to remove the m2 unit label and any text that comes after that.

In [37]:
print(th_no_missing_price['Surface area'].isnull().sum())

0


In [38]:
th_no_missing_price[['Surface area', 'extra SF text']] = th_no_missing_price['Surface area'].str.split(' m²', n=1, expand=True)

In [39]:
sf_text = th_no_missing_price['Surface area'].str.contains(pat='\.', case=False).sum()
print(sf_text)

0


In [40]:
# Check dtype of SF column and other columns
th_no_missing_price.dtypes

Address description       object
Price                    float64
Rooms                     object
Rooms detailed            object
Surface area              object
Bathrooms                 object
Floor                     object
House type                object
Total building floors    float64
Year of construction     float64
Condition                 object
Energy Efficiency         object
URL                       object
Rooms detailed total      object
Other room details        object
extra SF text             object
dtype: object

In [41]:
# Remove commas from the SF column
th_no_missing_price['Surface area'] = th_no_missing_price['Surface area'].str.replace(',', '')

Surface area seems clean and ready to be converted to a numerical column

In [42]:
# Convert surface area column to numerical column
th_no_missing_price['Surface area'] = th_no_missing_price['Surface area'].astype(float)

In [43]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,"3°, with lift",Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,"3 bedrooms, 2 others), 1 bathroom, kitchen diner",
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,"6°, with lift",Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,"1 bedroom, 1 other), 1 bathroom, kitchenette",
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,"2°, with lift",Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,"4 bedrooms, 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,"4°, with lift",Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,"2 bedrooms, 5 others), 3 bathrooms, kitchen diner",
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,"3°, with lift",Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,"2 bedrooms, 2 others), 2 bathrooms, kitchen diner",


Let's continue cleaning this dataset by cleaning the Floor column. Most values seem to have a number of floors followed by the '°' symbol. Let's use that to separate out the floor of the house, and the rest of the text can go into a new (potentially boolean) column called 'Lift'

In [44]:
th_no_missing_price[['Floor', 'Lift']] = th_no_missing_price['Floor'].str.split('°', n=1, expand=True)

In [45]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,"3 bedrooms, 2 others), 1 bathroom, kitchen diner",,", with lift"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,"1 bedroom, 1 other), 1 bathroom, kitchenette",,", with lift"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,"4 bedrooms, 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail,", with lift"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,"2 bedrooms, 5 others), 3 bathrooms, kitchen diner",,", with lift"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,"2 bedrooms, 2 others), 2 bathrooms, kitchen diner",,", with lift"


Looks better. Now let's make sure there are no other texts in this column, or other unwanted values. Also need to check for null values.

In [46]:
print(th_no_missing_price['Floor'].isnull().sum())

8


In [47]:
floor_remaining_text = th_no_missing_price['Floor'].str.contains(pat='[a-z]', case=False).sum()
print(floor_remaining_text)

401


In [48]:
remaining_floor_texts = th_no_missing_price[th_no_missing_price['Floor'].str.contains(pat='[a-z]', case=False) == True]


In [49]:
remaining_floor_texts.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift
12,"Apartment corso Regina Margherita 153, San Donato, Turin",229000.0,5,"5 (3 bedrooms, 2 others), 2 bathrooms, kitchen diner",100.0,2,"2 floors: Semi-basement, 4",Apartment | Full ownership | Medium property class,4.0,1950.0,Excellent / Refurbished,F,https://www.immobiliare.it/en/annunci/99924722/,5,"3 bedrooms, 2 others), 2 bathrooms, kitchen diner",- See detail,", with lift, with disabled access"
15,"Apartment corso Giacomo Matteotti 47, Cittadella, Turin",790000.0,5,"5 (3 bedrooms, 2 others), 2 bathrooms, kitchen diner",182.0,2,"2 floors: Basement (-1), 3",Apartment | Full ownership | Stately property class,5.0,1877.0,Excellent / Refurbished,E,https://www.immobiliare.it/en/annunci/99518778/,5,"3 bedrooms, 2 others), 2 bathrooms, kitchen diner",- See detail,", with lift"
20,"3-room flat corso Principe Eugenio 1, Cittadella, Turin",180000.0,3,"3 (2 bedrooms, 1 other), 1 bathroom, kitchen diner",95.0,NaN,"2 floors: Semi-basement, 2",Apartment | Full ownership | Stately property class,5.0,1969.0,To be refurbished,G,https://www.immobiliare.it/en/annunci/101155679/,3,"2 bedrooms, 1 other), 1 bathroom, kitchen diner",- See detail,", with lift"
24,"2-room flat via Gian Francesco Bellezia 5, Quadrilatero Romano, Turin",210000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchen diner",70.0,NaN,"3 floors: from Basement (-2) to Basement (-1), 2",Apartment | Full ownership | Stately property class,3.0,1800.0,Excellent / Refurbished,G,https://www.immobiliare.it/en/annunci/101413803/,2,"1 bedroom, 1 other), 1 bathroom, kitchen diner",- See detail,", with lift, with disabled access"
27,"Single family villa via Principessa Clotilde 82, San Donato, Turin",273000.0,5+,"5+ (3 bedrooms, 3 others), 2 bathrooms, kitchen diner",130.0,2,Ground floor,Single family villa | Full ownership | Stately property class,1.0,1900.0,To be refurbished,E,https://www.immobiliare.it/en/annunci/101244853/,5+,"3 bedrooms, 3 others), 2 bathrooms, kitchen diner",,None


It seems that the remaining text values in the Floor column are referring to multi-floor houses/apartments. This needs a bit further investigation, because it's not clear from the value if the place is multi-floor, and whether the number at the end indicates to which floor in the building the apartment starts, or where exactly it is situated. And I also need to understand what a basement is exactly, and how this can be consistent with the rest of the data. 

We'll leave finalizing Floor for later and move on to extracting further text from the rooms detailed column. We already separated total rooms from this column, now we will separate the number of bedrooms:

In [50]:
th_no_missing_price[['Bedrooms', 'Other room details']] = th_no_missing_price['Other room details'].str.split('bedrooms', n=1, expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift,Bedrooms
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,", 2 others), 1 bathroom, kitchen diner",,", with lift",3
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,None,,", with lift","1 bedroom, 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,", 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail,", with lift",4
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,", 5 others), 3 bathrooms, kitchen diner",,", with lift",2
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,", 2 others), 2 bathrooms, kitchen diner",,", with lift",2


It seems there are still values with bedroom in the new bedroom column. Let's check how many:

In [51]:
bed_text = th_no_missing_price['Bedrooms'].str.contains(pat='bedroom', case=False).sum()
print(bed_text)

588


Okay, the problem was that I used 'bedrooms' as the delimiter to split, but actually there are 588 rows with only 1 *bedroom*. So let's do it one more time with bedroom as the delimiter:

In [52]:
th_no_missing_price[['Bedrooms', 'Other room details 2']] = th_no_missing_price['Bedrooms'].str.split('bedroom', n=1, expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift,Bedrooms,Other room details 2
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,", 2 others), 1 bathroom, kitchen diner",,", with lift",3,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,None,,", with lift",1,", 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,", 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail,", with lift",4,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,", 5 others), 3 bathrooms, kitchen diner",,", with lift",2,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,", 2 others), 2 bathrooms, kitchen diner",,", with lift",2,None


Now I need to merge the two Other room details columns I now have. Let's create a new column with them just in case to check it goes well:

In [53]:
th_no_missing_price['Other room details combined'] = th_no_missing_price['Other room details'] + th_no_missing_price['Other room details 2']
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift,Bedrooms,Other room details 2,Other room details combined
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,", 2 others), 1 bathroom, kitchen diner",,", with lift",3,None,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,None,,", with lift",1,", 1 other), 1 bathroom, kitchenette",NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,", 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail,", with lift",4,None,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,", 5 others), 3 bathrooms, kitchen diner",,", with lift",2,None,NaN
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,", 2 others), 2 bathrooms, kitchen diner",,", with lift",2,None,NaN


That didn't work. I need to find a way to combine the two columns in a way that it can deal with None in one of them. Might need to use some conditionals.

In [54]:
# Try to fill the na values with fillna() routine
th_no_missing_price['Other room details combined'] = th_no_missing_price['Other room details'].fillna('') + th_no_missing_price['Other room details 2'].fillna('')
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,Other room details,extra SF text,Lift,Bedrooms,Other room details 2,Other room details combined
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,", 2 others), 1 bathroom, kitchen diner",,", with lift",3,None,", 2 others), 1 bathroom, kitchen diner"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,None,,", with lift",1,", 1 other), 1 bathroom, kitchenette",", 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,", 1 other), 1 bathroom, kitchen diner",| commercial 134.5 m² - See detail,", with lift",4,None,", 1 other), 1 bathroom, kitchen diner"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,", 5 others), 3 bathrooms, kitchen diner",,", with lift",2,None,", 5 others), 3 bathrooms, kitchen diner"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,", 2 others), 2 bathrooms, kitchen diner",,", with lift",2,None,", 2 others), 2 bathrooms, kitchen diner"


In [55]:
print(th_no_missing_price['Other room details combined'].isnull().sum())

0


That seems to have done it. Next thing is to delete the 'Other room details 2' and 'Other room details' columns, and then to find a way to extract the bathroom data from the new combined other text column.

In [56]:
# Drop unnecessary columns
th_no_missing_price = th_no_missing_price.drop('Other room details', axis=1)
th_no_missing_price = th_no_missing_price.drop('Other room details 2', axis=1)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,NaN,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", 2 others), 1 bathroom, kitchen diner"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,NaN,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", 1 other), 1 bathroom, kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,NaN,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", 1 other), 1 bathroom, kitchen diner"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,", 5 others), 3 bathrooms, kitchen diner"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,", 2 others), 2 bathrooms, kitchen diner"


In [57]:
# Split room details column by 'bathroom'
th_no_missing_price[['Bathrooms', 'Other room details combined']] = th_no_missing_price['Other room details combined'].str.split(' bathroom', n=1, expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,", 2 others), 1",3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", kitchen diner"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,", 1 other), 1",6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", kitchenette"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,", 1 other), 1",2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", kitchen diner"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,", 5 others), 3",4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,"s, kitchen diner"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,", 2 others), 2",3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,"s, kitchen diner"


Okay, we've split the number of bathrooms, where relevant. Now I will remove the fluff in the bathroom column. Let's split the column again, this time by the parenthesis which seems to come after 'others':

In [58]:
th_no_missing_price[['Fluff', 'Bathrooms']] = th_no_missing_price['Bathrooms'].str.split(')', n=1, expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,", 1",3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", kitchen diner",", 2 others"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,", 1",6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", kitchenette",", 1 other"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,", 1",2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", kitchen diner",", 1 other"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,", 3",4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,"s, kitchen diner",", 5 others"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,", 2",3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,"s, kitchen diner",", 2 others"


In [59]:
# Check if there is any other text in the bathroom column
bth_txt = th_no_missing_price['Bathrooms'].str.contains(pat='[a-z]', case=False).sum()
print(bth_txt)

8


In [60]:
bth_txts = th_no_missing_price['Bathrooms'].str.contains(pat='[a-z]', case=False)

In [61]:
texts = th_no_missing_price[bth_txts == True]
texts.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
551,"Apartment corso Siracusa 99, Santa Rita, Turin",355000.0,5,"5 (3 bedrooms, 2 others), kitchen diner",161.0,", kitchen diner",7,Apartment | Full ownership | Stately property class,9.0,1965.0,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/100897471/,5,,", with lift",3,None,", 2 others"
759,"2-room flat via Belfiore, San Salvario - Dante, Turin",160000.0,2,"2 (1 bedroom, 1 other), kitchen nook",65.0,", kitchen nook",Mezzanine,Apartment | Full ownership | Stately property class,5.0,1965.0,New / Under construction,C,https://www.immobiliare.it/en/annunci/101537879/,2,,None,1,None,", 1 other"
784,"3-room flat via Vandalino 82/30, Pozzo Strada, Turin",148000.0,3,"3 (2 bedrooms, 1 other), kitchenette",75.0,", kitchenette",6,Apartment | Full ownership | Medium property class,7.0,1960.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/98804516/,3,,", with lift",2,None,", 1 other"
1010,"Apartment via Sondrio 13, Don Bosco, Turin",340000.0,5+,"5+ (5 bedrooms, 3 others), kitchen diner",200.0,", kitchen diner",3,Apartment | Full ownership | Stately property class,5.0,1995.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99031372/,5+,,", with lift",5,None,", 3 others"
1033,"Apartment piazza Vittorio Veneto, Giardini Reali, Turin",810000.0,5+,"5+ (3 bedrooms, 3 others), kitchen diner",229.0,", kitchen diner",2,Apartment | Full ownership | Stately property class,5.0,1900.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101637221/,5+,,", with lift",3,None,", 3 others"


Alright, it seems that the remaining text values in the Bathrooms column are just those apartments which did not have the number of bathrooms in the description, but did have kitchen/diner. Let's get rid of these.

In [62]:
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.replace('[a-z]', '')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1761308412.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.replace('[a-z]', '')


In [63]:
bth_txt = th_no_missing_price['Bathrooms'].str.contains(pat='[a-z]', case=False).sum()
print(bth_txt)

0


In [64]:
bath_commas = th_no_missing_price['Bathrooms'].str.contains(pat=',', case=False).sum()
print(bath_commas)

1884


In [65]:
bth_com = th_no_missing_price[th_no_missing_price['Bathrooms'].str.contains(pat=',', case=False) == True]
bth_com.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,", 1",3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", kitchen diner",", 2 others"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,", 1",6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", kitchenette",", 1 other"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,", 1",2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", kitchen diner",", 1 other"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,", 3",4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,"s, kitchen diner",", 5 others"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,", 2",3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,"s, kitchen diner",", 2 others"


In [66]:
# Let's remove commas from the Bathroom column
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.replace(',', '')

In [67]:
# Let's also remove whitespace
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.strip()

In [68]:
th_no_missing_price['Bathrooms'].unique

<bound method Series.unique of 0       1
1       1
2       1
3       3
4       2
       ..
1995    1
1996    1
1997    1
1998    1
1999    2
Name: Bathrooms, Length: 1967, dtype: object>

In [69]:
th_no_missing_price.shape

(1967, 19)

In [70]:
empty_baths = th_no_missing_price['Bathrooms'].isnull().sum()
print(empty_baths)

83


In [71]:
numbers_baths = th_no_missing_price['Bathrooms'].str.contains(pat='[0-9]', case=False).sum()
print(numbers_baths)

1876


In [72]:
numbers_baths + empty_baths

1959

numbers and empty values don't add up, so there are still some values in the bathroom column that shouldn't be there

In [73]:
not_numbers_baths = th_no_missing_price[th_no_missing_price['Bathrooms'].str.contains(pat='[0-9]', case=False) == False]
value = not_numbers_baths.loc[551, 'Bathrooms']

In [74]:
type(value)

str

In [75]:
print(value)

In [76]:
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.strip()

In [77]:
empty_baths = th_no_missing_price['Bathrooms'].isnull().sum()
print(empty_baths)

83


In [78]:
not_numbers_baths = th_no_missing_price[th_no_missing_price['Bathrooms'].str.contains(pat='[0-9]', case=False) == False]
not_numbers_baths.shape

(8, 19)

Okay, so I discovered thanks to ChatGPT that empty strings are not the same as null values. So these 8 values are just empty strings. I can change these to NaN, and then finally finalize this by converting the column to numbers. But first I'll deal with values containing '+'

In [79]:
# Check how many values are left with X+ rooms
pluses = th_no_missing_price['Bathrooms'].str.contains(pat='\+', case=False).sum()
print(pluses)

31


In [80]:
pluses_df = th_no_missing_price[th_no_missing_price['Bathrooms'].str.contains(pat='\+', case=False) == True]
pluses_df.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
100,"Single family villa Strada Guido Volante 121, Colle della Maddalena, Turin",1550000.0,5+,"5+ (6 bedrooms, 11 others), 3+ bathrooms, kitchen diner",700.0,3+,Ground floor,Single family villa | Full ownership | Stately property class,3.0,1970.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/100355564/,5+,,None,6,"s, kitchen diner",", 11 others"
101,"Single family villa Strada dei Calleri 36, Superga, Turin",990000.0,5+,"5+ (5 bedrooms, 2 others), 3+ bathrooms, kitchen diner",820.0,3+,"3 floors: Basement (-1), Ground floor, 1",Single family villa | Full ownership | Stately property class,2.0,1950.0,Excellent / Refurbished,E,https://www.immobiliare.it/en/annunci/99558640/,5+,- See detail,", with lift, with disabled access",5,"s, kitchen diner",", 2 others"
138,"Penthouse piazza San Carlo, Via Roma, Turin",3400000.0,5+,"5+ (4 bedrooms, 4 others), 3+ bathrooms, kitchen diner",452.0,3+,"Multi-level, with lift",Penthouse | Full ownership | Stately property class,5.0,NaN,New / Under construction,G,https://www.immobiliare.it/en/annunci/87806794/,5+,,None,4,"s, kitchen diner",", 4 others"
157,"Apartment corso Giacomo Matteotti 42BIS, Cittadella, Turin",850000.0,5+,"5+ (5 bedrooms, 3 others), 3+ bathrooms, kitchen diner",280.0,3+,2,Apartment | Full ownership | Luxury property,9.0,1935.0,Good condition / Liveable,D,https://www.immobiliare.it/en/annunci/91215866/,5+,,", with lift",5,"s, kitchen diner",", 3 others"
213,"Single family villa Strada Comunale di Superga 322, Superga, Turin",698000.0,5+,"5+ (4 bedrooms, 8 others), 3+ bathrooms, kitchen diner",800.0,3+,"4 floors: from Semi-basement to Ground floor, from 1",Single family villa | Full ownership | Luxury property,4.0,2000.0,Excellent / Refurbished,C,https://www.immobiliare.it/en/annunci/100359380/,5+,- See detail,to 2°,4,"s, kitchen diner",", 8 others"


Okay, so there are 31 values of 3+ under bathrooms. I'm going to assume here that for all of these, there are at least 4 bathrooms:

In [81]:
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.replace(r'3\+', '4')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3047247391.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].str.replace(r'3\+', '4')


In [82]:
pluses = th_no_missing_price['Bathrooms'].str.contains(pat=r'\+', case=False).sum()
print(pluses)

0


And finally, we can convert bathrooms to a float column:

We convert bathrooms to a float column (for modeling later, needs to be a numerical data type). Side note: it turns out that you can use the replace method without str, in that case, replacing the empty string with a NaN DOES work. I had trouble with this in an earlier attempt because I used the str method.

In [83]:
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].replace('', np.nan)

In [84]:
# Convert bathrooms column to float and check how many missing values there are now
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].astype(float)
print(th_no_missing_price['Bathrooms'].isnull().sum())

91


In [85]:
th_no_missing_price.isnull().sum()

Address description              0
Price                            0
Rooms                           52
Rooms detailed                   0
Surface area                     0
Bathrooms                       91
Floor                            8
House type                       0
Total building floors           30
Year of construction           107
Condition                        0
Energy Efficiency              396
URL                              0
Rooms detailed total             0
extra SF text                    0
Lift                           286
Bedrooms                        83
Other room details combined     91
Fluff                            0
dtype: int64

In [86]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", kitchen diner",", 2 others"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", kitchenette",", 1 other"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", kitchen diner",", 1 other"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,"s, kitchen diner",", 5 others"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,"s, kitchen diner",", 2 others"


There's a bunch of columns now that I think I can delete. First I will check if there are any leftover numbers or other important details, before dropping them:

In [87]:
th_no_missing_price['Other room details combined'].str.contains(pat='[0-9]', case=False).sum()

0

In [88]:
no_kitch = th_no_missing_price[th_no_missing_price['Other room details combined'].str.contains(pat='kitchen', case=False) == False]
no_kitch.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Other room details combined,Fluff
122,"Apartment piazza Vittorio Veneto 18, Giardini Reali, Turin",580000.0,5+,"5+ (2 bedrooms, 4 others), 2 bathrooms",150.0,2.0,1,Apartment | Full ownership | Stately property class,4.0,NaN,Excellent / Refurbished,E,https://www.immobiliare.it/en/annunci/100850005/,5+,,"floor, with lift",2,s,", 4 others"
587,"3-room flat via Rovereto 58, Santa Rita, Turin",129000.0,3,"3 (1 bedroom, 2 others), 1 bathroom",80.0,1.0,2,Apartment | Full ownership,5.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/100402250/,3,,"floor, with lift",1,,", 2 others"
724,"Apartment corso Filippo Turati 14, San Secondo, Turin",475000.0,5+,"5+ (4 bedrooms, 3 others), 3 bathrooms",214.0,3.0,9,Apartment | Full ownership | Stately property class,9.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/99969668/,5+,,", with lift",4,s,", 3 others"
731,"Apartment via della Consolata, 15, Quadrilatero Romano, Turin",315000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom",120.0,1.0,1,Apartment | Full ownership | Medium property class,4.0,1870.0,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/99226920/,5,,", with lift, with disabled access",3,,", 2 others"
747,"3-room flat corso Regina 75, Vanchiglia, Turin",179000.0,3,"3 (1 bedroom, 2 others), 1 bathroom",60.0,1.0,NaN,Apartment,NaN,NaN,To be refurbished,NaN,https://www.immobiliare.it/en/annunci/101223573/,3,,NaN,1,,", 2 others"


In [89]:
no_kitch.shape

(24, 19)

In [90]:
no_kitch['Other room details combined'].isnull().sum()

0

In [91]:
# Drop unneccessary column
th_no_missing_price = th_no_missing_price.drop('Other room details combined', axis=1)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Fluff
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,", 2 others"
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,", 1 other"
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,", 1 other"
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,", 5 others"
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,", 2 others"


In [92]:
no_others = th_no_missing_price[th_no_missing_price['Fluff'].str.contains(pat='other', case=False) == False]

In [93]:
no_others.shape

(83, 18)

In [94]:
no_others.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Fluff
104,"Studio via Vittorio Andreis 4A, Aurora, Turin",42000.0,NaN,"1 bedroom, 1 bathroom, kitchen nook",30.0,NaN,Mezzanine,Apartment | Full ownership | Medium property class,4.0,1900.0,To be refurbished,E,https://www.immobiliare.it/en/annunci/99167344/,"1 bedroom, 1 bathroom, kitchen nook",,None,None,
125,"Apartment corso Vittorio Emanuele II 191, Cenisia, Turin",320000.0,5+,"6 bedrooms, 1 bathroom, kitchen diner",150.0,NaN,5,Apartment | Full ownership | Medium property class,5.0,NaN,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/100464900/,"6 bedrooms, 1 bathroom, kitchen diner",,", with lift",None,
169,Building for Sale,442000.0,5,"5 rooms, 3+ bathrooms",181.0,NaN,5,Building | Stately property class,1.0,1935.0,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/100803569/,"5 rooms, 3+ bathrooms",,,None,
205,"Studio via Cesare Balbo 22, Vanchiglia, Turin",49600.0,NaN,"1 bedroom, 1 bathroom, open kitchen",25.0,NaN,5,Apartment | Full ownership | Medium property class,5.0,1900.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/99962938/,"1 bedroom, 1 bathroom, open kitchen",,,None,
229,"Studio corso Corsica 28, Lingotto, Turin",30000.0,NaN,"1 bedroom, 1 bathroom, open kitchen",15.0,NaN,"Mezzanine, with lift",Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/98639484/,"1 bedroom, 1 bathroom, open kitchen",,None,None,


In [95]:
others = th_no_missing_price[th_no_missing_price['Fluff'].str.contains(pat='other', case=False) == True]

In [96]:
others.shape

(1884, 18)

In [97]:
1884+83

1967

Fluff only seems to have others text, or empty strings. So let's drop it:

In [98]:
# Drop fluff
th_no_missing_price = th_no_missing_price.drop('Fluff', axis=1)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2


Let's now clean up the lift column. First let's check how many 'with lift' rows there are:

In [99]:
th_no_missing_price['Lift'].str.contains(pat='with lift', case=False).sum()

1267

In [100]:
no_lifts = th_no_missing_price[th_no_missing_price['Lift'].str.contains(pat='with lift', case=False) == False]
no_lifts.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms
8,"4-room flat via Paolo Veronese 134/18, Barriera di Lanzo, Turin",149000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",101.0,2.0,1,Apartment | Full ownership | Medium property class,2.0,1960.0,Excellent / Refurbished,F,https://www.immobiliare.it/en/annunci/97609534/,4,,,2
33,"4-room flat corso Regina Margherita 121, Quadrilatero Romano, Turin",189000.0,4,"4 (3 bedrooms, 1 other), 1 bathroom, kitchen nook",70.0,1.0,3,Apartment | Full ownership | Medium property class,5.0,1900.0,Good condition / Liveable,D,https://www.immobiliare.it/en/annunci/101244343/,4,,,3
39,"4-room flat via Giambattista Bogino, 25, 10123, Via Roma, Turin",650000.0,4,"4 (3 bedrooms, 1 other), 1 bathroom, kitchen diner",177.0,1.0,1,Apartment | Full ownership,4.0,1800.0,To be refurbished,D,https://www.immobiliare.it/en/annunci/101386571/,4,,,3
41,"Apartment via Giambattista Bogino, 25, 10123, Via Roma, Turin",1040000.0,5,"5 (3 bedrooms, 2 others), 3 bathrooms, kitchen diner",275.0,3.0,1,Apartment | Full ownership,4.0,1800.0,To be refurbished,D,https://www.immobiliare.it/en/annunci/101386575/,5,,,3
42,"4-room flat via Salvatore Cognetti De Martiis 48, Lucento, Turin",95000.0,4,"4 (2 bedrooms, 2 others), 1 bathroom, kitchen diner",95.0,1.0,3,Apartment | Full ownership | Medium property class,3.0,1970.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/97629474/,4,,,2


In [101]:
no_lifts.shape

(414, 17)

In [102]:
no_lifts['Lift'].isnull().sum()

0

In [103]:
no_lifts_with_text = no_lifts['Lift'].str.contains(pat='[a-z]', case=False).sum()
print(no_lifts_with_text)

52


In [104]:
nlwt = no_lifts[no_lifts['Lift'].str.contains(pat='[a-z]', case=False)]
nlwt.head()


,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms
75,"4-room flat via Marco Polo, Crocetta, Turin",420000.0,4,"4 (2 bedrooms, 2 others), 1 bathroom, kitchen diner",110.0,1.0,4,Apartment | Full ownership | Stately property class,6.0,NaN,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/100850057/,4,,floor,2
135,"Two-family villa via Andrea Sansovino 108, Lucento, Turin",134000.0,4,"4 (2 bedrooms, 2 others), 1 bathroom, kitchen diner",93.0,1.0,1,Two-family villa | Full ownership | Medium property class,1.0,1950.0,Excellent / Refurbished,G,https://www.immobiliare.it/en/annunci/101544795/,4,,floor,2
143,"4-room flat via Passalacqua 3, Cittadella, Turin",399000.0,4,"4 (3 bedrooms, 1 other), 2 bathrooms, open kitchen",130.0,2.0,2,Apartment | Full ownership | Medium property class,2.0,1950.0,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/99927150/,4,,floor,3
213,"Single family villa Strada Comunale di Superga 322, Superga, Turin",698000.0,5+,"5+ (4 bedrooms, 8 others), 3+ bathrooms, kitchen diner",800.0,4.0,"4 floors: from Semi-basement to Ground floor, from 1",Single family villa | Full ownership | Luxury property,4.0,2000.0,Excellent / Refurbished,C,https://www.immobiliare.it/en/annunci/100359380/,5+,- See detail,to 2°,4
298,"Apartment via Milano, Quadrilatero Romano, Turin",229000.0,5,"5 (2 bedrooms, 3 others), 1 bathroom, kitchen diner",115.0,1.0,1,Apartment | Full ownership | Stately property class,6.0,1700.0,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/99257772/,5,,"floor, with disabled access",2


In [105]:
nlwtnf = nlwt[nlwt['Lift'].str.contains(pat='floor', case=False) == False]
nlwtnf.head()
nlwtnf.shape

(32, 17)

In [106]:
nlwtnf['Lift'].str.contains(pat='to', case=False).sum()

15

In [107]:
nlwtnf['Lift'].str.contains(pat='disabled', case=False).sum()

18

Okay, it seems that there are some listings (15) with a lift to a certain floor. I'm just going to count these as having a lift, even if it doesn't go to the floor of the apartment/listing. There are other listings that just have 'floor' under the lift column. I'm not sure if the web scraper picked up this field even though it was hidden, or where this is coming from, but i'm just going to assume that this means no floor. Ditto for 'with disabled access'. Actually, after looking at the csv, I see that the 'floor' text is just fluff from me splitting the description under "Floor" in the original listing. So indeed I can just remove the remaining values with only "floor" at the end. Looking at the CSV, I think I will also create another column for disabled access, because I think this is actually an important feature for some buyers, but also because there seems to be a good number of listings including this in the description.

In [108]:
# Check number of disabled access
da = th_no_missing_price['Lift'].str.contains(pat='disabled', case=False).sum()
print(da)

350


In [109]:
da = th_no_missing_price[th_no_missing_price['Lift'].str.contains(pat='disabled', case=False) == True]
da.head()


,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms
12,"Apartment corso Regina Margherita 153, San Donato, Turin",229000.0,5,"5 (3 bedrooms, 2 others), 2 bathrooms, kitchen diner",100.0,2.0,"2 floors: Semi-basement, 4",Apartment | Full ownership | Medium property class,4.0,1950.0,Excellent / Refurbished,F,https://www.immobiliare.it/en/annunci/99924722/,5,- See detail,", with lift, with disabled access",3
17,"Penthouse via Pavone 3, Borgo Vittoria, Turin",206000.0,3,"3 (2 bedrooms, 1 other), 1 bathroom, kitchenette",94.0,1.0,5,Penthouse | Full ownership | Stately property class,5.0,2003.0,Good condition / Liveable,D,https://www.immobiliare.it/en/annunci/99874804/,3,| commercial 115.5 m² - See detail,", with lift, with disabled access",2
24,"2-room flat via Gian Francesco Bellezia 5, Quadrilatero Romano, Turin",210000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchen diner",70.0,1.0,"3 floors: from Basement (-2) to Basement (-1), 2",Apartment | Full ownership | Stately property class,3.0,1800.0,Excellent / Refurbished,G,https://www.immobiliare.it/en/annunci/101413803/,2,- See detail,", with lift, with disabled access",1
30,"Apartment via Galliate, Parella, Turin",498000.0,5+,"5+ (3 bedrooms, 4 others), 2 bathrooms, kitchen diner",240.0,2.0,2,Apartment | Full ownership | Stately property class,3.0,1962.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/100106390/,5+,,", with lift, with disabled access",3
64,"3-room flat corso Rodolfo Montevecchio 35, Crocetta, Turin",770000.0,3,"3 (2 bedrooms, 1 other), 3 bathrooms, kitchen diner",265.0,3.0,3,Apartment | Full ownership | Stately property class,3.0,1967.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/87277810/,3,,", with lift, with disabled access",2


In [110]:
# Let's split disabled access into a new column
th_no_missing_price[['Lift', 'Disabled Access']] = th_no_missing_price['Lift'].str.split('with disabled ', expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,None
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,None


In [111]:
access_check = th_no_missing_price['Disabled Access'].str.contains(pat='access', case=False).sum()
print(access_check)

350


In [112]:
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.strip()
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,", with lift",3,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,", with lift",1,None
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,", with lift",4,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,", with lift",2,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,", with lift",2,None


In [113]:
# Replace 'with lift' and 'to' values with 'True'
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r', with lift', 'True')
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r'to.*', 'True')
th_no_missing_price.head()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\2312676498.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r'to.*', 'True')


,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1,None
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2,None


In [114]:
remaining_text = th_no_missing_price[th_no_missing_price['Lift'].str.contains(pat='True', case=False) == False]
len(remaining_text)

399

In [115]:
print(remaining_text['Lift'].str.contains(pat='[a-z]', case=False).sum())


20


In [116]:
az = remaining_text[remaining_text['Lift'].str.contains(pat='[a-z]', case=False) == True]
az.head(len(az))

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
75,"4-room flat via Marco Polo, Crocetta, Turin",420000.0,4,"4 (2 bedrooms, 2 others), 1 bathroom, kitchen diner",110.0,1.0,4,Apartment | Full ownership | Stately property class,6.0,NaN,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/100850057/,4,,floor,2,None
135,"Two-family villa via Andrea Sansovino 108, Lucento, Turin",134000.0,4,"4 (2 bedrooms, 2 others), 1 bathroom, kitchen diner",93.0,1.0,1,Two-family villa | Full ownership | Medium property class,1.0,1950.0,Excellent / Refurbished,G,https://www.immobiliare.it/en/annunci/101544795/,4,,floor,2,None
143,"4-room flat via Passalacqua 3, Cittadella, Turin",399000.0,4,"4 (3 bedrooms, 1 other), 2 bathrooms, open kitchen",130.0,2.0,2,Apartment | Full ownership | Medium property class,2.0,1950.0,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/99927150/,4,,floor,3,None
298,"Apartment via Milano, Quadrilatero Romano, Turin",229000.0,5,"5 (2 bedrooms, 3 others), 1 bathroom, kitchen diner",115.0,1.0,1,Apartment | Full ownership | Stately property class,6.0,1700.0,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/99257772/,5,,"floor,",2,access
664,"2-room flat via Ceresole 15, Barriera di Milano, Turin",45000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, open kitchen",50.0,1.0,5,Apartment | Full ownership | Medium property class,5.0,1930.0,Excellent / Refurbished,G,https://www.immobiliare.it/en/annunci/98380954/,2,,floor,1,None
792,"Apartment new, third floor, Gran Madre - Crimea, Turin",560000.0,5,"5 rooms, kitchen diner",145.0,NaN,3,Apartment,NaN,NaN,New / Under construction,NaN,https://www.immobiliare.it/en/annunci/101004053/,"5 rooms, kitchen diner",,floor,None,None
939,"3-room flat corso MONCALIERI 295, Cavoretto, Turin",157000.0,3,"3 (1 bedroom, 2 others), 1 bathroom, kitchen diner",65.0,1.0,1,Apartment | Full ownership | Medium property class,3.0,1950.0,Excellent / Refurbished,NaN,https://www.immobiliare.it/en/annunci/91703264/,3,,floor,1,None
1009,"2-room flat Verolengo 182, Lucento, Turin",65000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchen nook",60.0,1.0,2,Apartment | Full ownership | Stately property class,3.0,1950.0,Good condition / Liveable,NaN,https://www.immobiliare.it/en/annunci/94425828/,2,,floor,1,None
1200,"3-room flat via via BURONZO 3, Campidoglio, Turin",98000.0,3,"3 (2 bedrooms, 1 other), 1 bathroom, semi-habitable kitchen",76.0,1.0,2,Apartment | Full ownership | Medium property class,2.0,1930.0,Excellent / Refurbished,F,https://www.immobiliare.it/en/annunci/99964678/,3,,floor,2,None
1299,"3-room flat via San Domenico 37, Cittadella, Turin",160000.0,3,"3 (2 bedrooms, 1 other), 1 bathroom, kitchen diner",87.0,1.0,2,Apartment,NaN,NaN,To be refurbished,NaN,https://www.immobiliare.it/en/annunci/100921559/,3,,floor,2,None


In [117]:
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace('floor', 'False')

In [118]:
th_no_missing_price['Lift'].unique()

array(['True', '', 'True,', None, 'FalseTrue,', 'FalseTrue', 'False',
       'False,', ',', nan, ', 8°True', ', 6°True', ', 5°True,',
       ', 3°True'], dtype=object)

In [119]:
# Seems that there were some cases of 'floor' inside of values with 'to' or with lift
# Let's correct these to just True
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace('FalseTrue', 'False')
th_no_missing_price['Lift'].unique()

array(['True', '', 'True,', None, 'False,', 'False', ',', nan, ', 8°True',
       ', 6°True', ', 5°True,', ', 3°True'], dtype=object)

In [120]:
# And let's remove the floor number with '°'
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r'.*°', '')
th_no_missing_price['Lift'].unique()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\102946652.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r'.*°', '')


array(['True', '', 'True,', None, 'False,', 'False', ',', nan],
      dtype=object)

In [121]:
# And let's also remove the commas
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace(r',', '')
th_no_missing_price['Lift'].unique()

array(['True', '', None, 'False', nan], dtype=object)

In [122]:
# Let's fill nan values with empty strings, which presumably will be converted to False when we convert the entire column to a boolean column
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].fillna('')
th_no_missing_price['Lift'].unique()

array(['True', '', 'False'], dtype=object)

In [123]:
# Seems that astype() converts False strings to True, so just in case let's now replace False values with empty strings
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].str.replace('False', '')
th_no_missing_price['Lift'].value_counts()

True    1187
         780
Name: Lift, dtype: int64

In [124]:
# And finally let's convert the Lift column to a boolean column
th_no_missing_price['Lift'] = th_no_missing_price['Lift'].astype(bool)
th_no_missing_price['Lift'].value_counts()

True     1187
False     780
Name: Lift, dtype: int64

In [125]:
type(th_no_missing_price['Lift'][0])

numpy.bool_

In [126]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1,None
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2,None


In [127]:
# Let's also fillna the Disabled Access column to replace the None values
th_no_missing_price['Disabled Access'].unique()

array([None, 'access', nan], dtype=object)

In [128]:
th_no_missing_price['Disabled Access'] = th_no_missing_price['Disabled Access'].fillna('False')
th_no_missing_price['Disabled Access'].unique()

array(['False', 'access'], dtype=object)

In [129]:
th_no_missing_price['Disabled Access'].value_counts()

False     1617
access     350
Name: Disabled Access, dtype: int64

In [130]:
th_no_missing_price['Disabled Access'] = th_no_missing_price['Disabled Access'].str.replace('access', 'True')
th_no_missing_price['Disabled Access'].value_counts()

False    1617
True      350
Name: Disabled Access, dtype: int64

In [131]:
th_no_missing_price['Disabled Access'] = th_no_missing_price['Disabled Access'].str.strip()
# I've had trouble converting False to False, it seems all strings (including 'False') are converted to True except empty strings
th_no_missing_price['Disabled Access'] = th_no_missing_price['Disabled Access'].str.replace('False', '')
th_no_missing_price['Disabled Access'] = th_no_missing_price['Disabled Access'].astype(bool)
th_no_missing_price['Disabled Access'].value_counts()

False    1617
True      350
Name: Disabled Access, dtype: int64

In [132]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3,False
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1,False
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4,False
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2,False
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2,False


Okay, I think we are almost finished cleaning this dataset. Last things to do are:
1. Compare columns 'Rooms' and 'Rooms detailed total' to see whether there are gaps and whether there is a way to merge them when one of the columns is empty AND when the columns have the same values
2. Clean up the number columns that are still string columns, such as Bedrooms, Floor.
3. Clean up the Energy efficiency column.
4. Find some kind of categorization for the columns 'House type' and 'Condition'
5. Check the non-empty 'extra SF text' column to see if there is anything useful there, otherwise drop it
6. Deconstruct the address column, so that I can take out the neighborhood and use it as a separate column. OR, more interestingly, convert these addresses to coordinates.

In [133]:
# Combine the Rooms columns
th_no_missing_price['Rooms combined'] = th_no_missing_price['Rooms'].combine_first(th_no_missing_price['Rooms detailed total'])
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,Energy Efficiency,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access,Rooms combined
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,F,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3,False,5
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,F,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1,False,2
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,E,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4,False,5
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,D,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2,False,5+
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,G,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2,False,4


In [134]:
th_no_missing_price['Rooms combined'].value_counts()

3                                       632
4                                       456
2                                       434
5                                       227
5+                                      166
1 bedroom, 1 bathroom, kitchen nook      19
1 bedroom, 1 bathroom, open kitchen      14
1 bedroom, 1 bathroom, kitchenette        3
1 bedroom, 1 bathroom, kitchen diner      3
1 room, 1 bathroom, open kitchen          2
1 bathroom, open kitchen                  2
5+                                        2
1 room, 1 bathroom                        1
1 bathroom, kitchen diner                 1
2 bathrooms, kitchen diner                1
1 room, 1 bathroom, kitchen diner         1
2                                         1
3+ bathrooms, kitchen diner               1
1 room, 1 bathroom, kitchen nook          1
Name: Rooms combined, dtype: int64

So it seems that there are some rows with 5+ rooms, but in parenthesis it states how many there are exactly. So I'm going to try and extract that.

In [135]:
# Check how many rows there are with at least '5+'
count_plusses = th_no_missing_price[th_no_missing_price['Rooms detailed'].str.contains(pat='^5\+', case=False) == True]
print(len(count_plusses))
count_plusses['Rooms detailed'].value_counts()

167


5+ (3 bedrooms, 3 others), 2 bathrooms, kitchen diner                  26
5+ (4 bedrooms, 2 others), 2 bathrooms, kitchen diner                  12
5+ (3 bedrooms, 3 others), 3 bathrooms, kitchen diner                   9
5+ (3 bedrooms, 4 others), 3 bathrooms, kitchen diner                   9
5+ (3 bedrooms, 4 others), 2 bathrooms, kitchen diner                   7
                                                                       ..
5+ (20 bedrooms, 2 others), 3+ bathrooms, kitchen diner                 1
5+ (3 bedrooms, 5 others), 1 bathroom, semi-habitable kitchen           1
5+ (6 bedrooms, 1 other), 2 bathrooms, kitchen diner                    1
5+ (4 bedrooms, 3 others), 3 bathrooms                                  1
5+ (4 bedrooms, 2 others), 3 bathrooms, kitchen diner, tennis court     1
Name: Rooms detailed, Length: 76, dtype: int64

In [136]:
# Check how many rows there are with 5+ and other text
count_plusses = th_no_missing_price[th_no_missing_price['Rooms detailed'].str.contains(pat='\+ \(', case=False) == True]
print(len(count_plusses))


158


It seems there are some rows with 5+ that do not contain other text info about the exact number of rooms

In [137]:
# Check how many occurences there are of '5+' without further info with a parenthesis
only_5plus = th_no_missing_price[th_no_missing_price['Rooms detailed'].str.contains(pat=r'^(?!.*\().*5\+.*$', case=False) == True]
print(len(only_5plus))

9


In [138]:
count_plusses['Rooms detailed'].value_counts()

5+ (3 bedrooms, 3 others), 2 bathrooms, kitchen diner                  26
5+ (4 bedrooms, 2 others), 2 bathrooms, kitchen diner                  12
5+ (3 bedrooms, 3 others), 3 bathrooms, kitchen diner                   9
5+ (3 bedrooms, 4 others), 3 bathrooms, kitchen diner                   9
5+ (3 bedrooms, 4 others), 2 bathrooms, kitchen diner                   7
                                                                       ..
5+ (2 bedrooms, 5 others), 2 bathrooms, kitchen diner                   1
5+ (20 bedrooms, 2 others), 3+ bathrooms, kitchen diner                 1
5+ (3 bedrooms, 5 others), 1 bathroom, semi-habitable kitchen           1
5+ (6 bedrooms, 1 other), 2 bathrooms, kitchen diner                    1
5+ (4 bedrooms, 2 others), 3 bathrooms, kitchen diner, tennis court     1
Name: Rooms detailed, Length: 73, dtype: int64

In [139]:
# df['5+ rooms exact number'] = df['text'].str.extract('\((.*?)\)')
# This will extract the text within the parentheses
th_no_missing_price['5+ rooms exact number'] = count_plusses['Rooms detailed'].str.extract('\+ \((.*?)\)')

In [140]:
# Check extraction
th_no_missing_price['5+ rooms exact number'].head()

0                     NaN
1                     NaN
2                     NaN
3    2 bedrooms, 5 others
4                     NaN
Name: 5+ rooms exact number, dtype: object

In [141]:
# Split the extracted text to get separate numbers
th_no_missing_price[['5+ exact rooms bedrooms', '5+ exact rooms others']] = th_no_missing_price['5+ rooms exact number'].str.split(', ', expand=True)

In [142]:
# Check split
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,...,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3,False,5,NaN,NaN,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,...,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1,False,2,NaN,NaN,NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,...,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4,False,5,NaN,NaN,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2,False,5+,"2 bedrooms, 5 others",2 bedrooms,5 others
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,...,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2,False,4,NaN,NaN,NaN


In [143]:
# Extract numbers from the '5+ exact rooms bedrooms column
th_no_missing_price['5+ exact rooms bedrooms NUM'] = th_no_missing_price['5+ exact rooms bedrooms'].str.extract(r'(\d+)')

# Extract numbers from the '5+ exact rooms others' column
th_no_missing_price['5+ exact rooms others NUM'] = th_no_missing_price['5+ exact rooms others'].str.extract(r'(\d+)')

In [144]:
# Check extraction
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,extra SF text,Lift,Bedrooms,Disabled Access,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,...,,True,3,False,5,NaN,NaN,NaN,NaN,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,...,,True,1,False,2,NaN,NaN,NaN,NaN,NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,...,| commercial 134.5 m² - See detail,True,4,False,5,NaN,NaN,NaN,NaN,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,,True,2,False,5+,"2 bedrooms, 5 others",2 bedrooms,5 others,2,5
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,...,,True,2,False,4,NaN,NaN,NaN,NaN,NaN


In [145]:
# Convert columns to num before adding them
th_no_missing_price['5+ exact rooms bedrooms NUM'] = th_no_missing_price['5+ exact rooms bedrooms NUM'].astype(float)
th_no_missing_price['5+ exact rooms others NUM'] = th_no_missing_price['5+ exact rooms others NUM'].astype(float)

# Make final exact rooms column for 5+ rooms listings
th_no_missing_price['5+ exact rooms FINAL'] = th_no_missing_price['5+ exact rooms bedrooms NUM'] + th_no_missing_price['5+ exact rooms others NUM']

# Check final column for 5+ listings
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,Lift,Bedrooms,Disabled Access,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM,5+ exact rooms FINAL
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,...,True,3,False,5,NaN,NaN,NaN,NaN,NaN,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,...,True,1,False,2,NaN,NaN,NaN,NaN,NaN,NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,...,True,4,False,5,NaN,NaN,NaN,NaN,NaN,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,True,2,False,5+,"2 bedrooms, 5 others",2 bedrooms,5 others,2.0,5.0,7.0
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,...,True,2,False,4,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
# And finally we can combine the new exact number of rooms with the rest of the rooms column
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['5+ exact rooms FINAL'].combine_first(th_no_missing_price['Rooms combined'])
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,Bedrooms,Disabled Access,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM,5+ exact rooms FINAL,Rooms FINAL
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,...,3,False,5,NaN,NaN,NaN,NaN,NaN,NaN,5
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,...,1,False,2,NaN,NaN,NaN,NaN,NaN,NaN,2
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,...,4,False,5,NaN,NaN,NaN,NaN,NaN,NaN,5
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,2,False,5+,"2 bedrooms, 5 others",2 bedrooms,5 others,2.0,5.0,7.0,7.0
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,...,2,False,4,NaN,NaN,NaN,NaN,NaN,NaN,4


In [147]:
# Check values of new rooms column
th_no_missing_price['Rooms FINAL'].value_counts()

3                                       632
4                                       456
2                                       434
5                                       227
6.0                                      74
7.0                                      34
8.0                                      19
1 bedroom, 1 bathroom, kitchen nook      19
1 bedroom, 1 bathroom, open kitchen      14
5+                                       10
10.0                                      8
9.0                                       7
12.0                                      5
1 bedroom, 1 bathroom, kitchenette        3
11.0                                      3
1 bedroom, 1 bathroom, kitchen diner      3
22.0                                      2
1 bathroom, open kitchen                  2
1 room, 1 bathroom, open kitchen          2
1 room, 1 bathroom, kitchen diner         1
1 room, 1 bathroom                        1
1 bathroom, kitchen diner                 1
16.0                            

In [148]:
# Replace remaining rows with only '5+' with 6, because information on the actual number seems to be missing but 5+ is going to be at least 6 rooms
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].str.replace('5\+', '6')

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1607674660.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].str.replace('5\+', '6')


In [149]:
checks = th_no_missing_price[th_no_missing_price['Rooms FINAL'].str.contains(pat='bathroom', case=False) == True]
checks.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,Bedrooms,Disabled Access,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM,5+ exact rooms FINAL,Rooms FINAL
104,"Studio via Vittorio Andreis 4A, Aurora, Turin",42000.0,NaN,"1 bedroom, 1 bathroom, kitchen nook",30.0,NaN,Mezzanine,Apartment | Full ownership | Medium property class,4.0,1900.0,...,None,False,"1 bedroom, 1 bathroom, kitchen nook",NaN,NaN,NaN,NaN,NaN,NaN,"1 bedroom, 1 bathroom, kitchen nook"
205,"Studio via Cesare Balbo 22, Vanchiglia, Turin",49600.0,NaN,"1 bedroom, 1 bathroom, open kitchen",25.0,NaN,5,Apartment | Full ownership | Medium property class,5.0,1900.0,...,None,False,"1 bedroom, 1 bathroom, open kitchen",NaN,NaN,NaN,NaN,NaN,NaN,"1 bedroom, 1 bathroom, open kitchen"
229,"Studio corso Corsica 28, Lingotto, Turin",30000.0,NaN,"1 bedroom, 1 bathroom, open kitchen",15.0,NaN,"Mezzanine, with lift",Apartment | Full ownership | Medium property class,6.0,1960.0,...,None,False,"1 bedroom, 1 bathroom, open kitchen",NaN,NaN,NaN,NaN,NaN,NaN,"1 bedroom, 1 bathroom, open kitchen"
261,"Studio via boccherini 28, Rebaudengo, Turin",29900.0,NaN,"1 bedroom, 1 bathroom, kitchenette",39.0,NaN,"Ground floor, with lift",Apartment | Full ownership | Medium property class,6.0,1970.0,...,None,False,"1 bedroom, 1 bathroom, kitchenette",NaN,NaN,NaN,NaN,NaN,NaN,"1 bedroom, 1 bathroom, kitchenette"
362,"Studio via Carlo Alberto 31, Via Roma, Turin",245000.0,NaN,"1 bedroom, 1 bathroom, kitchen nook",45.0,NaN,"Ground floor, with lift, with disabled access",Apartment | Full ownership | Stately property class,4.0,1880.0,...,None,False,"1 bedroom, 1 bathroom, kitchen nook",NaN,NaN,NaN,NaN,NaN,NaN,"1 bedroom, 1 bathroom, kitchen nook"


In [150]:
checks['Rooms FINAL'].value_counts()

1 bedroom, 1 bathroom, kitchen nook     19
1 bedroom, 1 bathroom, open kitchen     14
1 bedroom, 1 bathroom, kitchenette       3
1 bedroom, 1 bathroom, kitchen diner     3
1 bathroom, open kitchen                 2
1 room, 1 bathroom, open kitchen         2
2 bathrooms, kitchen diner               1
3+ bathrooms, kitchen diner              1
1 room, 1 bathroom, kitchen diner        1
1 bathroom, kitchen diner                1
1 room, 1 bathroom                       1
1 room, 1 bathroom, kitchen nook         1
Name: Rooms FINAL, dtype: int64

In [151]:
# Split the remaining rows with bedroom, bathroom, kitchen info
th_no_missing_price[['leftover_bedrooms', 'leftover_bathrooms']] = checks['Rooms FINAL'].str.split(' bedroom, ', expand=True)
th_no_missing_price['leftover_bedrooms'].value_counts()

1                                    39
1 bathroom, open kitchen              2
1 room, 1 bathroom, open kitchen      2
2 bathrooms, kitchen diner            1
3+ bathrooms, kitchen diner           1
1 room, 1 bathroom, kitchen diner     1
1 bathroom, kitchen diner             1
1 room, 1 bathroom                    1
1 room, 1 bathroom, kitchen nook      1
Name: leftover_bedrooms, dtype: int64

In [152]:
# Check again if there are any 5+ values left
blah = th_no_missing_price[th_no_missing_price['Rooms FINAL'].str.contains(pat=r'^5\+$') == True]
print(len(blah))

0


'leftover_bedrooms' still contains bedroom info under '1 room' and bathroom information. Let's create another set of leftover columns that splits this info.

In [153]:
th_no_missing_price[['leftover_bedrooms2', 'leftover_bathrooms2']] = th_no_missing_price['leftover_bedrooms'].str.split(' room, ', expand=True)

In [154]:
th_no_missing_price['leftover_bedrooms2'].value_counts() 

1                              44
1 bathroom, open kitchen        2
2 bathrooms, kitchen diner      1
3+ bathrooms, kitchen diner     1
1 bathroom, kitchen diner       1
Name: leftover_bedrooms2, dtype: int64

In [155]:
ones = th_no_missing_price['leftover_bedrooms2'].str.match(r'^1$')
ones.value_counts()

True     44
False     5
Name: leftover_bedrooms2, dtype: int64

In [156]:
th_no_missing_price['leftover_bathrooms2'].value_counts() 

1 bathroom, open kitchen     2
1 bathroom, kitchen diner    1
1 bathroom                   1
1 bathroom, kitchen nook     1
Name: leftover_bathrooms2, dtype: int64

In [157]:
# Create a 3rd column for leftover bathroom info in leftover_bedrooms2
th_no_missing_price['lb3'] = th_no_missing_price['leftover_bedrooms2']
th_no_missing_price['lb3'] = th_no_missing_price['lb3'].str.replace(r'^1$', '')
th_no_missing_price['lb3'] = th_no_missing_price['lb3'].replace('', np.nan)

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3978319110.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['lb3'] = th_no_missing_price['lb3'].str.replace(r'^1$', '')


In [158]:
# Check 3rd column for leftover bathroom info
th_no_missing_price['lb3'].value_counts()

1 bathroom, open kitchen       2
2 bathrooms, kitchen diner     1
3+ bathrooms, kitchen diner    1
1 bathroom, kitchen diner      1
Name: lb3, dtype: int64

In [159]:
# Combine lb3 with left_overbathrooms2
th_no_missing_price['leftover_bathrooms2'] = th_no_missing_price['leftover_bathrooms2'].combine_first(th_no_missing_price['lb3'])
th_no_missing_price['leftover_bathrooms2'].value_counts()

1 bathroom, open kitchen       4
1 bathroom, kitchen diner      2
2 bathrooms, kitchen diner     1
3+ bathrooms, kitchen diner    1
1 bathroom                     1
1 bathroom, kitchen nook       1
Name: leftover_bathrooms2, dtype: int64

In [160]:
# Replace the '3+' bathrooms value with '4'
th_no_missing_price['leftover_bathrooms2'] = th_no_missing_price['leftover_bathrooms2'].str.replace(r'3\+', '4')
th_no_missing_price['leftover_bathrooms2'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\2160077822.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['leftover_bathrooms2'] = th_no_missing_price['leftover_bathrooms2'].str.replace(r'3\+', '4')


1 bathroom, open kitchen      4
1 bathroom, kitchen diner     2
2 bathrooms, kitchen diner    1
4 bathrooms, kitchen diner    1
1 bathroom                    1
1 bathroom, kitchen nook      1
Name: leftover_bathrooms2, dtype: int64

In [161]:
th_no_missing_price['leftover_bathrooms'].value_counts()

1 bathroom, kitchen nook     19
1 bathroom, open kitchen     14
1 bathroom, kitchenette       3
1 bathroom, kitchen diner     3
Name: leftover_bathrooms, dtype: int64

In [162]:
# Combine leftover_bathrooms and leftover_bathrooms2
th_no_missing_price['leftover_bathrooms'] = th_no_missing_price['leftover_bathrooms'].combine_first(th_no_missing_price['leftover_bathrooms2'])
th_no_missing_price['leftover_bathrooms'].value_counts()

1 bathroom, kitchen nook      20
1 bathroom, open kitchen      18
1 bathroom, kitchen diner      5
1 bathroom, kitchenette        3
2 bathrooms, kitchen diner     1
4 bathrooms, kitchen diner     1
1 bathroom                     1
Name: leftover_bathrooms, dtype: int64

In [163]:
# Get rid of text in the leftover_bathrooms column
th_no_missing_price['leftover_bathrooms'] = th_no_missing_price['leftover_bathrooms'].str.extract(r'^(\d+)')
th_no_missing_price['leftover_bathrooms'].value_counts()

1    47
2     1
4     1
Name: leftover_bathrooms, dtype: int64

In [164]:
# And finally let's convert 'leftover_bathrooms' to a float so that we can combine it with the Bathrooms column
th_no_missing_price['leftover_bathrooms']= th_no_missing_price['leftover_bathrooms'].astype(float)

In [165]:
th_no_missing_price['leftover_bathrooms'].value_counts()

1.0    47
2.0     1
4.0     1
Name: leftover_bathrooms, dtype: int64

In [166]:
th_no_missing_price['Bathrooms'].value_counts()

1.0    1234
2.0     521
3.0      90
4.0      31
Name: Bathrooms, dtype: int64

In [167]:
# Combine Bathrooms and leftover_bathrooms
th_no_missing_price['Bathrooms'] = th_no_missing_price['Bathrooms'].combine_first(th_no_missing_price['leftover_bathrooms'])
th_no_missing_price['Bathrooms'].value_counts()


1.0    1281
2.0     522
3.0      90
4.0      32
Name: Bathrooms, dtype: int64

In [168]:
# Done with Bathrooms! Now let's drop the leftover bathroom columns
th_no_missing_price = th_no_missing_price.drop('leftover_bathrooms', axis=1)
th_no_missing_price = th_no_missing_price.drop('leftover_bathrooms2', axis=1)
th_no_missing_price = th_no_missing_price.drop('lb3', axis=1)

In [169]:
th_no_missing_price.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM,5+ exact rooms FINAL,Rooms FINAL,leftover_bedrooms,leftover_bedrooms2
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,5,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,...,5,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,2,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,...,2,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,5,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,...,5,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,5+,"2 bedrooms, 5 others",2 bedrooms,5 others,2.0,5.0,7.0,NaN,NaN,NaN
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,4,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,...,4,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN


In [170]:
# Check that we still have the Bedrooms column
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms', 'Rooms detailed',
       'Surface area', 'Bathrooms', 'Floor', 'House type',
       'Total building floors', 'Year of construction', 'Condition',
       'Energy Efficiency', 'URL', 'Rooms detailed total', 'extra SF text',
       'Lift', 'Bedrooms', 'Disabled Access', 'Rooms combined',
       '5+ rooms exact number', '5+ exact rooms bedrooms',
       '5+ exact rooms others', '5+ exact rooms bedrooms NUM',
       '5+ exact rooms others NUM', '5+ exact rooms FINAL', 'Rooms FINAL',
       'leftover_bedrooms', 'leftover_bedrooms2'],
      dtype='object')

In [171]:
th_no_missing_price['leftover_bedrooms2'].value_counts()

1                              44
1 bathroom, open kitchen        2
2 bathrooms, kitchen diner      1
3+ bathrooms, kitchen diner     1
1 bathroom, kitchen diner       1
Name: leftover_bedrooms2, dtype: int64

In [172]:
# Replace rows with bathroom and other text with None
th_no_missing_price['leftover_bedrooms2'] = np.where(th_no_missing_price['leftover_bedrooms2'].str.contains(r'\D'), None, th_no_missing_price['leftover_bedrooms2'])
th_no_missing_price['leftover_bedrooms2'].value_counts()

1    44
Name: leftover_bedrooms2, dtype: int64

In [173]:
th_no_missing_price['leftover_bedrooms'].value_counts()

1                                    39
1 bathroom, open kitchen              2
1 room, 1 bathroom, open kitchen      2
2 bathrooms, kitchen diner            1
3+ bathrooms, kitchen diner           1
1 room, 1 bathroom, kitchen diner     1
1 bathroom, kitchen diner             1
1 room, 1 bathroom                    1
1 room, 1 bathroom, kitchen nook      1
Name: leftover_bedrooms, dtype: int64

In [174]:
# Replace rows with bathroom + other text with None
th_no_missing_price['leftover_bedrooms'] = np.where(th_no_missing_price['leftover_bedrooms'].str.contains(r'\D'), None, th_no_missing_price['leftover_bedrooms'])
th_no_missing_price['leftover_bedrooms'].value_counts()

1    39
Name: leftover_bedrooms, dtype: int64

In [175]:
# Combine leftover_bedrooms and leftover_bedrooms2
th_no_missing_price['leftover_bedrooms'] = th_no_missing_price['leftover_bedrooms'].combine_first(th_no_missing_price['leftover_bedrooms2'])
th_no_missing_price['leftover_bedrooms'].value_counts()


1    44
Name: leftover_bedrooms, dtype: int64

In [176]:
th_no_missing_price['leftover_bedrooms'] = th_no_missing_price['leftover_bedrooms'].astype(float)
th_no_missing_price['leftover_bedrooms'].value_counts()

1.0    44
Name: leftover_bedrooms, dtype: int64

In [177]:
th_no_missing_price['5+ exact rooms bedrooms NUM'].value_counts()

3.0     71
4.0     42
5.0     15
2.0      9
6.0      6
10.0     4
8.0      3
9.0      3
7.0      2
26.0     1
20.0     1
1.0      1
Name: 5+ exact rooms bedrooms NUM, dtype: int64

In [178]:
# Combine '5+ exact rooms bedrooms NUM' and leftover_bedrooms
th_no_missing_price['5+ exact rooms bedrooms NUM'] = th_no_missing_price['5+ exact rooms bedrooms NUM'].combine_first(th_no_missing_price['leftover_bedrooms'])
th_no_missing_price['5+ exact rooms bedrooms NUM'].value_counts()

3.0     71
1.0     45
4.0     42
5.0     15
2.0      9
6.0      6
10.0     4
8.0      3
9.0      3
7.0      2
26.0     1
20.0     1
Name: 5+ exact rooms bedrooms NUM, dtype: int64

In [179]:
th_no_missing_price['Bedrooms'].value_counts()

2      862
1      588
3      332
4       67
5       15
6        6
10       4
8        3
9        3
7        2
26       1
20       1
Name: Bedrooms, dtype: int64

In [180]:
th_no_missing_price['Bedrooms'] = th_no_missing_price['Bedrooms'].astype(float)
th_no_missing_price['Bedrooms'].value_counts()

2.0     862
1.0     588
3.0     332
4.0      67
5.0      15
6.0       6
10.0      4
8.0       3
9.0       3
7.0       2
26.0      1
20.0      1
Name: Bedrooms, dtype: int64

In [181]:
# Combine Bedrooms and '5+ exact rooms bedrooms NUM'
th_no_missing_price['Bedrooms'] = th_no_missing_price['Bedrooms'].combine_first(th_no_missing_price['5+ exact rooms bedrooms NUM'])
th_no_missing_price['Bedrooms'].value_counts()

2.0     862
1.0     632
3.0     332
4.0      67
5.0      15
6.0       6
10.0      4
8.0       3
9.0       3
7.0       2
26.0      1
20.0      1
Name: Bedrooms, dtype: int64

In [182]:
th_no_missing_price['Rooms FINAL'].value_counts()

3                                       632
4                                       456
2                                       434
5                                       227
1 bedroom, 1 bathroom, kitchen nook      19
1 bedroom, 1 bathroom, open kitchen      14
6                                        10
1 bedroom, 1 bathroom, kitchenette        3
1 bedroom, 1 bathroom, kitchen diner      3
1 room, 1 bathroom, open kitchen          2
1 bathroom, open kitchen                  2
1 room, 1 bathroom                        1
1 bathroom, kitchen diner                 1
2 bathrooms, kitchen diner                1
1 room, 1 bathroom, kitchen diner         1
2                                         1
3+ bathrooms, kitchen diner               1
1 room, 1 bathroom, kitchen nook          1
Name: Rooms FINAL, dtype: int64

In [183]:
# Replace 'Rooms FINAL' rows with bathroom + other text with None
th_no_missing_price['Rooms FINAL'] = np.where(th_no_missing_price['Rooms FINAL'].str.contains(r'\D'), None, th_no_missing_price['Rooms FINAL'])
th_no_missing_price['Rooms FINAL'].value_counts()

3    632
4    456
2    434
5    227
6     10
Name: Rooms FINAL, dtype: int64

In [184]:
# Convert 'Rooms FINAL' to float
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].astype(float)
th_no_missing_price['Rooms FINAL'].value_counts()

3.0    632
4.0    456
2.0    434
5.0    227
6.0     10
Name: Rooms FINAL, dtype: int64

In [185]:
# Add leftover_bedrooms to Rooms FINAL, and this should include those that were leftover after splitting the info in parentheses of cases of rooms with 5+ info
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].combine_first(th_no_missing_price['leftover_bedrooms'])
th_no_missing_price['Rooms FINAL'].value_counts()

3.0    632
4.0    456
2.0    434
5.0    227
1.0     44
6.0     10
Name: Rooms FINAL, dtype: int64

In [186]:
th_no_missing_price['Rooms combined'].value_counts()

3                                       632
4                                       456
2                                       434
5                                       227
5+                                      166
1 bedroom, 1 bathroom, kitchen nook      19
1 bedroom, 1 bathroom, open kitchen      14
1 bedroom, 1 bathroom, kitchenette        3
1 bedroom, 1 bathroom, kitchen diner      3
1 room, 1 bathroom, open kitchen          2
1 bathroom, open kitchen                  2
5+                                        2
1 room, 1 bathroom                        1
1 bathroom, kitchen diner                 1
2 bathrooms, kitchen diner                1
1 room, 1 bathroom, kitchen diner         1
2                                         1
3+ bathrooms, kitchen diner               1
1 room, 1 bathroom, kitchen nook          1
Name: Rooms combined, dtype: int64

In [187]:
# Replace 'Rooms combined' rows with text with None
th_no_missing_price['Rooms combined'] = np.where(th_no_missing_price['Rooms combined'].str.contains(r'\D'), None, th_no_missing_price['Rooms combined'])
th_no_missing_price['Rooms combined'].value_counts()

3    632
4    456
2    434
5    227
Name: Rooms combined, dtype: int64

In [188]:
th_no_missing_price['Rooms combined'] = th_no_missing_price['Rooms combined'].astype(float)

In [189]:
# Add 'Rooms combined' to Rooms FINAL, and this should finally include all rooms, 
# including those that I combined at the beginning of the whole rooms data cleaning
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].combine_first(th_no_missing_price['Rooms combined'])
th_no_missing_price['Rooms FINAL'].value_counts()

3.0    632
4.0    456
2.0    434
5.0    227
1.0     44
6.0     10
Name: Rooms FINAL, dtype: int64

Okay, I think we are finished cleaning the rooms information. The only thing I can think of that might still be potentially missing info is the bathroom information in the cases where there was 5+ rooms with bedroom info in parentheses. I'm not sure if in those cases the bathroom info is already included in the bathroom column. I could potentially check this by filtering the data and looking at the bathroom column when rooms detailed has 5+.

In [190]:
# Filter Rooms detailed on '5+' and check if 'Bathrooms' info is missing
fiveplus_condition = th_no_missing_price['Rooms detailed'].str.contains(pat=r'\+ \(', case=False) == True

# Use Boolean indexing to retrieve values from the 'Bathrooms' column based on the filter
filtered_bathrooms = th_no_missing_price.loc[fiveplus_condition, 'Bathrooms']

filtered_bathrooms.value_counts()

2.0    70
3.0    48
4.0    29
1.0     7
Name: Bathrooms, dtype: int64

In [191]:
filtered_bathrooms_df = th_no_missing_price[fiveplus_condition]
filtered_bathrooms_df.head()

,Address description,Price,Rooms,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,...,Rooms combined,5+ rooms exact number,5+ exact rooms bedrooms,5+ exact rooms others,5+ exact rooms bedrooms NUM,5+ exact rooms others NUM,5+ exact rooms FINAL,Rooms FINAL,leftover_bedrooms,leftover_bedrooms2
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,5+,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,...,NaN,"2 bedrooms, 5 others",2 bedrooms,5 others,2.0,5.0,7.0,NaN,NaN,None
19,"Apartment corso Monte Cucco, 81, Pozzo Strada, Turin",315000.0,5+,"5+ (4 bedrooms, 2 others), 2 bathrooms, kitchen diner",180.0,2.0,1,Apartment | Full ownership | Stately property class,10.0,1971.0,...,NaN,"4 bedrooms, 2 others",4 bedrooms,2 others,4.0,2.0,6.0,NaN,NaN,None
27,"Single family villa via Principessa Clotilde 82, San Donato, Turin",273000.0,5+,"5+ (3 bedrooms, 3 others), 2 bathrooms, kitchen diner",130.0,2.0,Ground floor,Single family villa | Full ownership | Stately property class,1.0,1900.0,...,NaN,"3 bedrooms, 3 others",3 bedrooms,3 others,3.0,3.0,6.0,NaN,NaN,None
30,"Apartment via Galliate, Parella, Turin",498000.0,5+,"5+ (3 bedrooms, 4 others), 2 bathrooms, kitchen diner",240.0,2.0,2,Apartment | Full ownership | Stately property class,3.0,1962.0,...,NaN,"3 bedrooms, 4 others",3 bedrooms,4 others,3.0,4.0,7.0,NaN,NaN,None
40,"Apartment via Ugo Foscolo 30, San Salvario - Dante, Turin",415000.0,5+,"5+ (3 bedrooms, 3 others), 3 bathrooms, kitchen diner",188.0,3.0,1,Apartment | Full ownership | Stately property class,5.0,1939.0,...,NaN,"3 bedrooms, 3 others",3 bedrooms,3 others,3.0,3.0,6.0,NaN,NaN,None


In [192]:
# Save the filtered dataframe to a CSV to see whether the bathroom info is in another column for those 4 values
filtered_bathrooms_df.to_csv('filtered_5+_bathrooms.csv', index=False)

After checking the null Bathroom values for rows with the filter for 5+, I see that the rooms information did not go through properly into the Rooms column. I need to investigate this.

In [193]:
# It seems I forgot to include the '5+ exact rooms FINAL' column
th_no_missing_price['Rooms FINAL'] = th_no_missing_price['Rooms FINAL'].combine_first(th_no_missing_price['5+ exact rooms FINAL'])
th_no_missing_price['Rooms FINAL'].value_counts()

3.0     632
4.0     456
2.0     434
5.0     227
6.0      84
1.0      44
7.0      34
8.0      19
10.0      8
9.0       7
12.0      5
11.0      3
22.0      2
17.0      1
52.0      1
20.0      1
15.0      1
13.0      1
16.0      1
Name: Rooms FINAL, dtype: int64

In [194]:
# Let's check if those missing Rooms are now in the dataframe. One more time:

# Filter Rooms detailed on '5+' and check if 'Bathrooms' info is missing
fiveplus_condition = th_no_missing_price['Rooms detailed'].str.contains(pat=r'\+ \(', case=False) == True

# Use Boolean indexing to retrieve values from the 'Bathrooms' column based on the filter
filtered_bathrooms = th_no_missing_price.loc[fiveplus_condition, 'Bathrooms']
filtered_bathrooms_df = th_no_missing_price[fiveplus_condition]

# Save the filtered dataframe to a CSV to see whether the bathroom info is in another column for those 4 values
filtered_bathrooms_df.to_csv('filtered_5+_bathrooms2.csv', index=False)

In [195]:
filtered_bathrooms_df['Rooms FINAL'].isnull().sum()

0

Yes! No blank values left now for rows when Bathroom info is missing.
Last things to do are:
1. Drop columns i no longer need (Rooms Final is the final room column, Bathrooms and Bedrooms are the final bath/bed columns)
2. Clean up the number columns that are still string columns, such as Bedrooms, Floor.
3. Clean up the Energy efficiency column.
4. Find some kind of categorization for the columns 'House type' and 'Condition'
5. Check the non-empty 'extra SF text' column to see if there is anything useful there, otherwise drop it
6. Deconstruct the address column, so that I can take out the neighborhood and use it as a separate column. OR, more interestingly, convert these addresses to coordinates.

In [196]:
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms', 'Rooms detailed',
       'Surface area', 'Bathrooms', 'Floor', 'House type',
       'Total building floors', 'Year of construction', 'Condition',
       'Energy Efficiency', 'URL', 'Rooms detailed total', 'extra SF text',
       'Lift', 'Bedrooms', 'Disabled Access', 'Rooms combined',
       '5+ rooms exact number', '5+ exact rooms bedrooms',
       '5+ exact rooms others', '5+ exact rooms bedrooms NUM',
       '5+ exact rooms others NUM', '5+ exact rooms FINAL', 'Rooms FINAL',
       'leftover_bedrooms', 'leftover_bedrooms2'],
      dtype='object')

In [197]:
# Drop redundant bathroom, bedroom and room columns
th_no_missing_price = th_no_missing_price.drop('leftover_bedrooms', axis=1)
th_no_missing_price = th_no_missing_price.drop('leftover_bedrooms2', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ exact rooms FINAL', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ exact rooms others NUM', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ exact rooms bedrooms NUM', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ exact rooms others', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ exact rooms bedrooms', axis=1)
th_no_missing_price = th_no_missing_price.drop('5+ rooms exact number', axis=1)
th_no_missing_price = th_no_missing_price.drop('Rooms combined', axis=1)
# Let's keep 'Rooms detailed' for now just to check at the end that there isn't any missing info that is actually available
# th_no_missing_price = th_no_missing_price.drop('Rooms detailed', axis=1)
th_no_missing_price = th_no_missing_price.drop('Rooms', axis=1)

In [198]:
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL'],
      dtype='object')

In [199]:
th_no_missing_price['Condition'].value_counts()

Excellent / Refurbished      769
Good condition / Liveable    763
To be refurbished            260
New / Under construction     175
Name: Condition, dtype: int64

In [200]:
th_no_missing_price['Condition'].isnull().sum()

0

'Condition' thankfully has 4 discrete and consistent categories. We can one-hot encode these when it comes time to build a model.

In [201]:
th_no_missing_price['Floor'].value_counts()

1                                                              358
2                                                              299
3                                                              290
4                                                              227
5                                                              161
                                                              ... 
2 floors: Semi-basement, 4                                       1
2 floors: Basement (-1), Ground floor, with disabled access      1
2 floors: Basement (-1), 13                                      1
2 floors: Basement (-1), 7                                       1
2 floors: Ground floor, 4                                        1
Name: Floor, Length: 88, dtype: int64

In [202]:
# Save the df to CSV to try to understand what is meant by the 2 floors entries and other numbers under the 'Floors' column
th_no_missing_price.to_csv('floors.csv', index=False)

So it seems that the floor column has some rows with '2 floors:...' After thinking about what this means, I realized it implies there are indeed two floors, either the apartment itself is two floors, or the apartment has a storage of some kind on a lower floor. What I will do is filter these two floor entries, and then split them where there are two different floors, or just use the floor that the two room apartment begins at.

Okay, there are actually more cases of multiple floors. In the case of two, it was clear this was usually a basement level (probably storage). In 3 or more, it seems to just be a multistory house, large apartment or entire building. So I'm not sure what to do with this data. Because the way I have interpreted this column is "the floor the apartment is located in" but the cases of three or more are implying instead "the number of floors it has". There are also of course cases of 'Ground floor', which I guess I will just include as '0'.

I think that the most logical way to deal with this is to use the Floor column as 'the floor the home is in'when the 'House Type' is 'Apartment' or 'Loft'. The other types of house, like a house or entire building which is on the ground, can just be '0'.

In [203]:
th_no_missing_price['House type'].value_counts()

Apartment | Full ownership | Medium property class                        924
Apartment | Full ownership | Stately property class                       582
Apartment | Full ownership | Economic property class                      148
Apartment | Full ownership | Luxury property                               50
Apartment | Full ownership                                                 49
Penthouse | Full ownership | Stately property class                        24
Penthouse | Full ownership | Medium property class                         18
Apartment                                                                  17
Loft | Full ownership | Stately property class                             13
Apartment | Bare ownership | Medium property class                         12
Single family villa | Full ownership | Stately property class              12
Apartment | Stately property class                                         12
Loft | Full ownership | Medium property class                   

I think the House type data needs to be cleaned first before I can proceed with cleaning the floor data. There are multiple labels in each entry, and I think it might be worthwhile to use the first entry (probably the most important) as well as the last entry (if available) on the property class. I think this class categorization may imply some valuable information about the value of the property or the building, given there are 'economic', 'medium', 'stately' and 'luxury' classes. The full ownership doesn't seem to me to provide that much additional valuable information about the listing.

In [204]:
# Let's start by removing '| Full ownership' from the column, if it's in a given entry
th_no_missing_price['House type2'] = th_no_missing_price['House type'].str.replace('\| Full ownership', '', case=False)
th_no_missing_price['House type2'] = th_no_missing_price['House type2'].str.replace('\| Bare ownership', '', case=False)
th_no_missing_price['House type2'] = th_no_missing_price['House type2'].str.replace('\| Partial ownership', '', case=False)


th_no_missing_price['House type2'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3365191931.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['House type2'] = th_no_missing_price['House type'].str.replace('\| Full ownership', '', case=False)
C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3365191931.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['House type2'] = th_no_missing_price['House type2'].str.replace('\| Bare ownership', '', case=False)
C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\3365191931.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['House type2'] = th_no_missing_price['House type2'].str.replace('\| Partial ownership', '', case=False)


Apartment  | Medium property class                        936
Apartment  | Stately property class                       587
Apartment  | Economic property class                      148
Apartment  | Luxury property                               50
Apartment                                                  49
Penthouse  | Stately property class                        24
Penthouse  | Medium property class                         18
Apartment                                                  17
Loft  | Stately property class                             13
Loft  | Medium property class                              12
Apartment | Stately property class                         12
Single family villa  | Stately property class              12
Apartment | Medium property class                          11
Single family villa  | Medium property class                6
Attic  | Medium property class                              6
Attic  | Stately property class                             5
Building

In [205]:
# Save the column to csv just to double check that there aren't any other ownership categories
th_no_missing_price['House type2'].to_csv('house_types.csv', index=False)

Okay, it seems there's no more ownership labels. So I will now split the house type using '|' as a delimiter, so that we have the house type in one column and the property class (if available) in the other

In [206]:
# Split the 'house_type2' column to have the house type and property class as different columns
th_no_missing_price[['House_type_final', 'Property_class']] = th_no_missing_price['House type2'].str.split(pat=r'\|', n=1, expand=True)
th_no_missing_price.head()

,Address description,Price,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,...,URL,Rooms detailed total,extra SF text,Lift,Bedrooms,Disabled Access,Rooms FINAL,House type2,House_type_final,Property_class
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,...,https://www.immobiliare.it/en/annunci/99929544/,5,,True,3.0,False,5.0,Apartment | Medium property class,Apartment,Medium property class
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,...,https://www.immobiliare.it/en/annunci/97514484/,2,,True,1.0,False,2.0,Apartment | Medium property class,Apartment,Medium property class
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,...,https://www.immobiliare.it/en/annunci/101585695/,5,| commercial 134.5 m² - See detail,True,4.0,False,5.0,Apartment | Stately property class,Apartment,Stately property class
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,...,https://www.immobiliare.it/en/annunci/99936782/,5+,,True,2.0,False,7.0,Apartment | Stately property class,Apartment,Stately property class
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,...,https://www.immobiliare.it/en/annunci/92280220/,4,,True,2.0,False,4.0,Apartment | Stately property class,Apartment,Stately property class


In [207]:
th_no_missing_price['House_type_final'].value_counts()

Apartment                         1721
Apartment                           74
Penthouse                           43
Loft                                28
Single family villa                 21
Apartment                           17
Attic                               12
Building                            12
Single-family detached house         8
Open space                           6
Penthouse                            5
Multi-family detached house          5
Attic                                2
Two-family villa                     2
Single family villa                  2
Dairy farm                           1
Penthouse                            1
Single-family detached house         1
Farmhouse                            1
Multi-family villa                   1
Country house                        1
Loft                                 1
Terraced house                       1
Apartment in villa                   1
Name: House_type_final, dtype: int64

In [208]:
# Remove whitespace from 'House type final' column
th_no_missing_price['House_type_final'] = th_no_missing_price['House_type_final'].str.strip()
th_no_missing_price['House_type_final'].value_counts()

Apartment                       1812
Penthouse                         49
Loft                              29
Single family villa               23
Attic                             14
Building                          12
Single-family detached house       9
Open space                         6
Multi-family detached house        5
Two-family villa                   2
Dairy farm                         1
Farmhouse                          1
Multi-family villa                 1
Country house                      1
Terraced house                     1
Apartment in villa                 1
Name: House_type_final, dtype: int64

Some of these categories are maybe redundant. I think I can combine some under one common category but I need to investigate exactly what the difference is between a loft and an apartment for example, or a single family detached house and a single family villa.

In [209]:
# Save df to csv so I look up these listings with the URL to determine what exactly is a villa, detached house, loft etc., 
# with the intention of hopefully being able to combine some of these categories
th_no_missing_price[['House_type_final', 'URL']].to_csv('house_type_checks.csv', index=False)

Okay, there is clearly a big difference between a villa and a detached house. A detached house seems to include city houses that are semi detached, only one side is attached to another house. A villa on the other hand is indeed a large impressive country house. A loft is a type of apartment, but it seems important to distinguish it as a separate category because it's usually a larger open space or has some kind of unique design that sets it apart from a conventional apartment. An attic is the living space directly under the roof. We'll keep all these categories. Things to change though:

1. The open spaces to me look like they could either be group under apartment or loft - i think loft is better since they are quite unique layouts. 
2. The farmhouse and dairyfarm can just be Farmhouse
3. Country House maybe can stay? There is indeed a difference between a farm house (which is large and originally made for storage/animals) and a country house which is made for humans.
4. Two family villa we will change to multi family villa
5. Building seems to indeed be buildings or large estates with multiple floors or structures.

In [210]:
th_no_missing_price['House_type_final'] = th_no_missing_price['House_type_final'].str.replace('Open space', 'Loft', case=False)
th_no_missing_price['House_type_final'] = th_no_missing_price['House_type_final'].str.replace('Dairy farm', 'Farmhouse', case=False)
th_no_missing_price['House_type_final'] = th_no_missing_price['House_type_final'].str.replace('Two-family villa', 'Multi-family villa', case=False)
th_no_missing_price['House_type_final'].value_counts()

Apartment                       1812
Penthouse                         49
Loft                              35
Single family villa               23
Attic                             14
Building                          12
Single-family detached house       9
Multi-family detached house        5
Multi-family villa                 3
Farmhouse                          2
Country house                      1
Terraced house                     1
Apartment in villa                 1
Name: House_type_final, dtype: int64

Looks good! Now let's move on to cleaning up the new 'property class' column.

In [211]:
th_no_missing_price['Property_class'].value_counts()

 Medium property class                1012
 Stately property class                668
 Economic property class               156
 Luxury property                        58
 Usufruct | Stately property class       1
Name: Property_class, dtype: int64

It's already super clean! Because the 'usufruct' category of ownership only occurs once, and also because I already removed the ownership category because I considered it not really providing useful information for the model, I will remove it so that this row will just become part of the stately property class.

In [212]:
th_no_missing_price['Property_class'] = th_no_missing_price['Property_class'].str.replace('Usufruct \| ', '')
th_no_missing_price['Property_class'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1996244333.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['Property_class'] = th_no_missing_price['Property_class'].str.replace('Usufruct \| ', '')


 Medium property class      1012
 Stately property class      669
 Economic property class     156
 Luxury property              58
Name: Property_class, dtype: int64

Done! Now we can finally handle the floor information. Let's set up a few filter conditions to check the different values for house type categories.

In [213]:
# Create filter conditions for house_type categories to see what the floor values are under each category
apartment_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Apartment', case=False) == True
penthouse_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Penthouse', case=False) == True
loft_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Loft', case=False) == True
attic_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Attic', case=False) == True


In [214]:
# Use Boolean indexing to retrieve values from the 'Floor' column based on the filter
# Check floor values of each category
filtered_floors = th_no_missing_price.loc[apartment_condition, 'Floor']
filtered_floors.value_counts()



1                                                              346
2                                                              298
3                                                              283
4                                                              213
5                                                              153
                                                              ... 
2 floors: from 8                                                 1
2 floors: Semi-basement, 8                                       1
2 floors: from 5                                                 1
2 floors: Basement (-1), Ground floor, with disabled access      1
2 floors: Ground floor, 4                                        1
Name: Floor, Length: 68, dtype: int64

Okay, there are a number of rows that have two floors in the apartment category. Let's check how many and also if there are other rows in the apartment category with more than 2 floors.

In [215]:
(filtered_floors.str.contains(pat='2 floors', case=False) == True).sum() # 73
(filtered_floors.str.contains(pat='3 floors', case=False) == True).sum() # 15
(filtered_floors.str.contains(pat='4 floors', case=False) == True).sum() # 3
(filtered_floors.str.contains(pat='5 floors', case=False) == True).sum() # 0
(filtered_floors.str.contains(pat='6 floors', case=False) == True).sum() # 0
(filtered_floors.str.contains(pat='floors', case=False) == True).sum() # 91

91

So there are 18 cases of more than 2 floors. Earlier before dealing the house type column I had discovered that this column is not consistent, because some of the rows indicate the floor that the apartment is on, but others indicate the floors that the apartment includes (such as storage in the basement). I'm going to save the df and look at it more closely in a csv.

In [216]:
apartment_floors = th_no_missing_price[apartment_condition]
apartment_floors.to_csv('apartment_floors.csv', index=False)

Alright, so indeed it seems that the cases of '2 floors' either have a) a basement and the floor of the apartment or b) the floor that the two floor apartment begins at. The situation is similar for '3 floors' and '4 floors', but they either have more basements or more floors. So I think the way I'm going to approach this is to split this column by comma's, which will separate out the 'X floors: basement' info from the rest of the floors. I can then synthesize the floor information.

In [217]:
th_no_missing_price[['apartment_floors', 'apartment_floors2', 'apartment_floors3', 
                     'apartment_floors4']] = apartment_floors['Floor'].str.split(
                         ',', expand=True
                     )
th_no_missing_price.head()

,Address description,Price,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,...,Bedrooms,Disabled Access,Rooms FINAL,House type2,House_type_final,Property_class,apartment_floors,apartment_floors2,apartment_floors3,apartment_floors4
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,...,3.0,False,5.0,Apartment | Medium property class,Apartment,Medium property class,3,None,None,None
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,...,1.0,False,2.0,Apartment | Medium property class,Apartment,Medium property class,6,None,None,None
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,...,4.0,False,5.0,Apartment | Stately property class,Apartment,Stately property class,2,None,None,None
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,...,2.0,False,7.0,Apartment | Stately property class,Apartment,Stately property class,4,None,None,None
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,...,2.0,False,4.0,Apartment | Stately property class,Apartment,Stately property class,3,None,None,None


In [218]:
th_no_missing_price['apartment_floors'].value_counts()

1                                                346
2                                                298
3                                                283
4                                                213
5                                                153
Mezzanine                                        118
Ground floor                                      96
6                                                 92
7                                                 49
2 floors: Basement (-1)                           35
9                                                 29
8                                                 26
2 floors: Semi-basement                           15
2 floors: Ground floor                             9
3 floors: Basement (-1)                            8
10                                                 8
Multi-level                                        4
2 floors: from 1                                   3
2 floors: Basement (-2)                       

Looks like some floors just had the 'from X' info, so this didn't split with a comma. Let's split these as well using 'from' as the split.

In [219]:
th_no_missing_price[['apartment_floors', 'apartment_floors5']] = th_no_missing_price['apartment_floors'].str.split(r' from ', expand=True)
th_no_missing_price['apartment_floors'].value_counts()

1                          346
2                          298
3                          283
4                          213
5                          153
Mezzanine                  118
Ground floor                96
6                           92
7                           49
2 floors: Basement (-1)     35
9                           29
8                           26
2 floors: Semi-basement     15
2 floors:                   10
2 floors: Ground floor       9
10                           8
3 floors: Basement (-1)      8
Multi-level                  4
3 floors:                    3
3 floors: Ground floor       2
4 floors:                    2
2 floors: Basement (-2)      2
3 floors: Semi-basement      2
2 floors: 4                  1
4 floors: Basement (-1)      1
12                           1
Basement (-1)                1
2 floors: 1                  1
Name: apartment_floors, dtype: int64

In [220]:
th_no_missing_price['apartment_floors5'].value_counts()

Basement (-2) to Basement (-1)    3
1                                 3
9                                 2
3                                 1
8                                 1
5                                 1
4                                 1
Semi-basement to Ground floor     1
Basement (-1) to Semi-basement    1
2                                 1
Name: apartment_floors5, dtype: int64

In [221]:
th_no_missing_price['apartment_floors'].value_counts()

1                          346
2                          298
3                          283
4                          213
5                          153
Mezzanine                  118
Ground floor                96
6                           92
7                           49
2 floors: Basement (-1)     35
9                           29
8                           26
2 floors: Semi-basement     15
2 floors:                   10
2 floors: Ground floor       9
10                           8
3 floors: Basement (-1)      8
Multi-level                  4
3 floors:                    3
3 floors: Ground floor       2
4 floors:                    2
2 floors: Basement (-2)      2
3 floors: Semi-basement      2
2 floors: 4                  1
4 floors: Basement (-1)      1
12                           1
Basement (-1)                1
2 floors: 1                  1
Name: apartment_floors, dtype: int64

In [222]:
# Clean up 'apartment_floors' to remove irrelevant floor info and standardize
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace('with lift', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace('with disabled access', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'(\d+) floors: ', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace('Ground floor', '0')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace('Mezzanine', '0.5')
th_no_missing_price['apartment_floors'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1616036780.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'(\d+) floors: ', '')


1                347
2                298
3                283
4                214
5                153
0.5              118
0                107
6                 92
7                 49
Basement (-1)     45
9                 29
8                 26
Semi-basement     17
2 floors:         10
10                 8
Multi-level        4
3 floors:          3
Basement (-2)      2
4 floors:          2
12                 1
Name: apartment_floors, dtype: int64

In [223]:
# Remove remaining basement rows
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'Basement(.*)', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'Semi-basement(.*)', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'Multi-level', '')
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r' ', '')

th_no_missing_price['apartment_floors'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1479406936.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'Basement(.*)', '')
C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1479406936.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].str.replace(r'Semi-basement(.*)', '')


1           347
2           298
3           283
4           214
5           153
0.5         118
0           107
6            92
             68
7            49
9            29
8            26
2floors:     10
10            8
3floors:      3
4floors:      2
12            1
Name: apartment_floors, dtype: int64

In [224]:
# Rename 'apartment_floors5' to 'apartment_floors_from' to keep track of the 'from' information
th_no_missing_price = th_no_missing_price.rename(columns={'apartment_floors5': 'apartment_floors_from'})
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL', 'House type2', 'House_type_final',
       'Property_class', 'apartment_floors', 'apartment_floors2',
       'apartment_floors3', 'apartment_floors4', 'apartment_floors_from'],
      dtype='object')

In [225]:
th_no_missing_price['apartment_floors2'].value_counts()

 with lift               119
 with disabled access     34
 1                        18
 2                        17
 3                         9
 4                         8
 Ground floor              7
 Mezzanine                 5
 5                         4
 from 1                    2
 6                         2
 from 4                    1
 13                        1
 from 3                    1
 8                         1
 from 5                    1
 11                        1
 from 2                    1
Name: apartment_floors2, dtype: int64

In [226]:
# Now let's split the 'apartment_floors2' column using 'from' to get the from info
th_no_missing_price[['apartment_floors2', 'apartment_floors_from2']] = th_no_missing_price['apartment_floors2'].str.split('from ', expand=True)
th_no_missing_price['apartment_floors2'].value_counts()

 with lift               119
 with disabled access     34
 1                        18
 2                        17
 3                         9
 4                         8
 Ground floor              7
                           6
 Mezzanine                 5
 5                         4
 6                         2
 13                        1
 8                         1
 11                        1
Name: apartment_floors2, dtype: int64

In [227]:
th_no_missing_price['apartment_floors_from2'].value_counts()

1    2
3    1
2    1
4    1
5    1
Name: apartment_floors_from2, dtype: int64

In [228]:
# And let's clean up 'apartment_floors2'
th_no_missing_price['apartment_floors2'] = th_no_missing_price['apartment_floors2'].str.replace('with lift', '')
th_no_missing_price['apartment_floors2'] = th_no_missing_price['apartment_floors2'].str.replace('with disabled access', '')
th_no_missing_price['apartment_floors2'] = th_no_missing_price['apartment_floors2'].str.replace('Ground floor', '0')
th_no_missing_price['apartment_floors2'] = th_no_missing_price['apartment_floors2'].str.replace('Mezzanine', '0.5')
th_no_missing_price['apartment_floors2'].value_counts()

        159
 1       18
 2       17
 3        9
 4        8
 0        7
 0.5      5
 5        4
 6        2
 13       1
 8        1
 11       1
Name: apartment_floors2, dtype: int64

In [229]:
th_no_missing_price['apartment_floors3'].value_counts()

 with disabled access    46
 1                        5
 with lift                2
 from 1                   1
 Mezzanine                1
Name: apartment_floors3, dtype: int64

In [230]:
# There's one row with from info, so let's just remove the from text
th_no_missing_price['apartment_floors3'] = th_no_missing_price['apartment_floors3'].str.replace('from ', '')
th_no_missing_price['apartment_floors3'].value_counts()

 with disabled access    46
 1                        6
 with lift                2
 Mezzanine                1
Name: apartment_floors3, dtype: int64

In [231]:
# And let's also clean up 'apartment_floors3'
th_no_missing_price['apartment_floors3'] = th_no_missing_price['apartment_floors3'].str.replace('with lift', '')
th_no_missing_price['apartment_floors3'] = th_no_missing_price['apartment_floors3'].str.replace('with disabled access', '')
th_no_missing_price['apartment_floors3'] = th_no_missing_price['apartment_floors3'].str.replace('Mezzanine', '0.5')
th_no_missing_price['apartment_floors3'].value_counts()

        48
 1       6
 0.5     1
Name: apartment_floors3, dtype: int64

In [232]:
th_no_missing_price['apartment_floors4'].value_counts()

 with lift    1
Name: apartment_floors4, dtype: int64

In [233]:
# There's no useful data in 'apartment_floors4' so let's drop it
th_no_missing_price = th_no_missing_price.drop('apartment_floors4', axis=1)
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL', 'House type2', 'House_type_final',
       'Property_class', 'apartment_floors', 'apartment_floors2',
       'apartment_floors3', 'apartment_floors_from', 'apartment_floors_from2'],
      dtype='object')

In [234]:
th_no_missing_price['apartment_floors_from'].value_counts()

Basement (-2) to Basement (-1)    3
1                                 3
9                                 2
3                                 1
8                                 1
5                                 1
4                                 1
Semi-basement to Ground floor     1
Basement (-1) to Semi-basement    1
2                                 1
Name: apartment_floors_from, dtype: int64

In [235]:
# Let's also clean up 'apartment_floors_from' by removing remaining basement rows
th_no_missing_price['apartment_floors_from'] = th_no_missing_price['apartment_floors_from'].str.replace(r'Basement(.*)', '')
th_no_missing_price['apartment_floors_from'] = th_no_missing_price['apartment_floors_from'].str.replace(r'Semi-basement(.*)', '0')
th_no_missing_price['apartment_floors_from'].value_counts()

C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1971768552.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['apartment_floors_from'] = th_no_missing_price['apartment_floors_from'].str.replace(r'Basement(.*)', '')
C:\Users\Hans\AppData\Local\Temp\ipykernel_18800\1971768552.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  th_no_missing_price['apartment_floors_from'] = th_no_missing_price['apartment_floors_from'].str.replace(r'Semi-basement(.*)', '0')


     4
1    3
9    2
3    1
8    1
5    1
4    1
0    1
2    1
Name: apartment_floors_from, dtype: int64

In [236]:
th_no_missing_price['apartment_floors_from2'].value_counts()

1    2
3    1
2    1
4    1
5    1
Name: apartment_floors_from2, dtype: int64

In [237]:
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL', 'House type2', 'House_type_final',
       'Property_class', 'apartment_floors', 'apartment_floors2',
       'apartment_floors3', 'apartment_floors_from', 'apartment_floors_from2'],
      dtype='object')

In [238]:
# And finally, let's put all these columns together
th_no_missing_price['apartment_floors_from'] = th_no_missing_price['apartment_floors_from'].combine_first(th_no_missing_price['apartment_floors_from2'])
th_no_missing_price['apartment_floors3'] = th_no_missing_price['apartment_floors3'].combine_first(th_no_missing_price['apartment_floors_from'])
th_no_missing_price['apartment_floors2'] = th_no_missing_price['apartment_floors2'].combine_first(th_no_missing_price['apartment_floors3'])
th_no_missing_price['apartment_floors'] = th_no_missing_price['apartment_floors'].combine_first(th_no_missing_price['apartment_floors2'])
th_no_missing_price['apartment_floors'].value_counts()

1           347
2           298
3           283
4           214
5           153
0.5         118
0           107
6            92
             68
7            49
9            29
8            26
2floors:     10
10            8
3floors:      3
4floors:      2
12            1
Name: apartment_floors, dtype: int64

Okay, the floor data is clean for the apartment listings. Now I need to clean up the rest of the floor column for all the other house types. Let's see what that data looks like. 

In [239]:
# Use Boolean indexing to retrieve values from the 'Floor' column based on the filter
# Check floor values of each category
filtered_floors = th_no_missing_price.loc[penthouse_condition, 'Floor']
filtered_floors.value_counts()

4                                               9
6                                               6
5                                               5
8                                               4
7                                               3
3                                               3
2 floors: from 5                                2
2 floors: Ground floor, 2                       1
3 floors: Basement (-1), from 1                 1
2 floors: Ground floor, 3                       1
2 floors: Basement (-2), 6                      1
1                                               1
2 floors: Basement (-1), 7                      1
Top floor, with lift                            1
2 floors: Basement (-1), 4                      1
10                                              1
2 floors: Semi-basement, 6                      1
2 floors: Basement (-2), 1                      1
2 floors: from 6                                1
9                                               1


Similarly to what we did with the apartment info, let's split the penthouse floor info into different columns using a comma as the delimiter. Then we can similarly split the remaining values with 'from X' into yet another column. Then we can consolidate the remaining data.

In [241]:
# Save df to double check values in a csv
penthouse_floors = th_no_missing_price[penthouse_condition]
penthouse_floors.to_csv('penthouse_floors.csv', index=False)

In [243]:
# Split the penthouse-filtered floor column 
th_no_missing_price[['penthouse_floors', 'penthouse_floors2', 
                     'penthouse_floors3']] = penthouse_floors['Floor'].str.split(
                         ',', expand=True
                     )
th_no_missing_price.head()

,Address description,Price,Rooms detailed,Surface area,Bathrooms,Floor,House type,Total building floors,Year of construction,Condition,...,House_type_final,Property_class,apartment_floors,apartment_floors2,apartment_floors3,apartment_floors_from,apartment_floors_from2,penthouse_floors,penthouse_floors2,penthouse_floors3
0,"Apartment via Albenga 2, Lingotto, Turin",165000.0,"5 (3 bedrooms, 2 others), 1 bathroom, kitchen diner",100.0,1.0,3,Apartment | Full ownership | Medium property class,9.0,1960.0,To be refurbished,...,Apartment,Medium property class,3,None,None,None,None,NaN,NaN,NaN
1,"2-room flat via Duino 184BIS, Mirafiori Sud - Onorato Vigliani, Turin",71000.0,"2 (1 bedroom, 1 other), 1 bathroom, kitchenette",60.0,1.0,6,Apartment | Full ownership | Medium property class,6.0,1960.0,Good condition / Liveable,...,Apartment,Medium property class,6,None,None,None,None,NaN,NaN,NaN
2,"Apartment corso siracusa, 79, Santa Rita, Turin",235000.0,"5 (4 bedrooms, 1 other), 1 bathroom, kitchen diner",128.0,1.0,2,Apartment | Full ownership | Stately property class,7.0,1969.0,Good condition / Liveable,...,Apartment,Stately property class,2,None,None,None,None,NaN,NaN,NaN
3,"Apartment via XX SETTEMBRE, 9, Via Roma, Turin",735000.0,"5+ (2 bedrooms, 5 others), 3 bathrooms, kitchen diner",235.0,3.0,4,Apartment | Full ownership | Stately property class,4.0,1910.0,Excellent / Refurbished,...,Apartment,Stately property class,4,None,None,None,None,NaN,NaN,NaN
4,"4-room flat via STELVIO 67, Pozzo Strada, Turin",248000.0,"4 (2 bedrooms, 2 others), 2 bathrooms, kitchen diner",145.0,2.0,3,Apartment | Full ownership | Stately property class,7.0,1972.0,Good condition / Liveable,...,Apartment,Stately property class,3,None,None,None,None,NaN,NaN,NaN


In [244]:
# Check values in the first penthouse column
th_no_missing_price['penthouse_floors'].value_counts()

4                          9
6                          6
5                          5
8                          4
7                          3
3                          3
2 floors: Basement (-1)    3
2 floors: from 5           2
2 floors: Ground floor     2
2 floors: Basement (-2)    2
Multi-level                2
2 floors: Semi-basement    1
10                         1
2 floors: from 6           1
9                          1
Top floor                  1
1                          1
3 floors: Basement (-1)    1
2 floors: from 9           1
Name: penthouse_floors, dtype: int64

In [245]:
# Split remaining rows with 'from X' floor info
th_no_missing_price[['penthouse_floors', 'penthouse_floors4']] = th_no_missing_price['penthouse_floors'].str.split('from ', expand=True)

In [246]:
# Check values in the first penthouse column
th_no_missing_price['penthouse_floors'].value_counts()

4                          9
6                          6
5                          5
2 floors:                  4
8                          4
2 floors: Basement (-1)    3
7                          3
3                          3
Multi-level                2
2 floors: Basement (-2)    2
2 floors: Ground floor     2
9                          1
10                         1
2 floors: Semi-basement    1
Top floor                  1
1                          1
3 floors: Basement (-1)    1
Name: penthouse_floors, dtype: int64

In [247]:
# Let's clean up the 'penthouse_floors' column to remove irrelevant floor info and standardize
th_no_missing_price['penthouse_floors'] = th_no_missing_price['penthouse_floors'].str.replace('Top floor', '5') # in the CSV we saw that this value is 5 (total building floors)
th_no_missing_price['penthouse_floors'] = th_no_missing_price['penthouse_floors'].str.replace(r'(^.*\bfloors\b.*$)', '', regex=True) # Replace strings that have 'basement with empty string' 
th_no_missing_price['penthouse_floors'] = th_no_missing_price['penthouse_floors'].str.replace('Multi-level', '')
th_no_missing_price['penthouse_floors'].value_counts()

      15
4      9
5      6
6      6
8      4
7      3
3      3
10     1
9      1
1      1
Name: penthouse_floors, dtype: int64

In [248]:
# Check values in the 'penthouse_floors2' column
th_no_missing_price['penthouse_floors2'].value_counts()

 with lift    3
 6            2
 5            1
 1            1
 4            1
 2            1
 7            1
 3            1
 from 1       1
Name: penthouse_floors2, dtype: int64

In [250]:
# Let's clean up the 'penthouse_floors2' column to remove irrelevant floor info and standardize
th_no_missing_price['penthouse_floors2'] = th_no_missing_price['penthouse_floors2'].str.replace('with lift', '')
th_no_missing_price['penthouse_floors2'] = th_no_missing_price['penthouse_floors2'].str.replace('from 1', '1')

th_no_missing_price['penthouse_floors2'].value_counts()

      3
 1    2
 6    2
 5    1
 4    1
 2    1
 7    1
 3    1
Name: penthouse_floors2, dtype: int64

In [252]:
# Check 'penthouse_floors3' values
th_no_missing_price['penthouse_floors3'].value_counts()

 with disabled access    1
Name: penthouse_floors3, dtype: int64

In [255]:
# We can drop 'penthouse_floors3' since there's only 1 value which is redundant
th_no_missing_price.drop('penthouse_floors3', axis=1, inplace=True)
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL', 'House type2', 'House_type_final',
       'Property_class', 'apartment_floors', 'apartment_floors2',
       'apartment_floors3', 'apartment_floors_from', 'apartment_floors_from2',
       'penthouse_floors', 'penthouse_floors2', 'penthouse_floors4'],
      dtype='object')

In [257]:
# Check 'penthouse_floors4' values
th_no_missing_price['penthouse_floors4'].value_counts()

5    2
6    1
9    1
Name: penthouse_floors4, dtype: int64

In [258]:
# And finally, let's put all these penthouse columns together
th_no_missing_price['penthouse_floors2'] = th_no_missing_price['penthouse_floors2'].combine_first(th_no_missing_price['penthouse_floors4'])
th_no_missing_price['penthouse_floors'] = th_no_missing_price['penthouse_floors'].combine_first(th_no_missing_price['penthouse_floors2'])
th_no_missing_price['penthouse_floors'].value_counts()

      15
4      9
5      6
6      6
8      4
7      3
3      3
10     1
9      1
1      1
Name: penthouse_floors, dtype: int64

In [259]:
# Let's drop redundant columns now that we used to clean up apartments and penthouses
columns_to_drop = ['apartment_floors2','apartment_floors3', 'apartment_floors_from', 'apartment_floors_from2',
                    'penthouse_floors2', 'penthouse_floors4']
th_no_missing_price.drop(columns_to_drop, axis=1, inplace=True)
th_no_missing_price.columns

Index(['Address description', 'Price', 'Rooms detailed', 'Surface area',
       'Bathrooms', 'Floor', 'House type', 'Total building floors',
       'Year of construction', 'Condition', 'Energy Efficiency', 'URL',
       'Rooms detailed total', 'extra SF text', 'Lift', 'Bedrooms',
       'Disabled Access', 'Rooms FINAL', 'House type2', 'House_type_final',
       'Property_class', 'apartment_floors', 'penthouse_floors'],
      dtype='object')

Alright, now that apartments and penthouses are clean, lets move on to the next house types and their floor values.

In [260]:
th_no_missing_price['House_type_final'].value_counts()

Apartment                       1812
Penthouse                         49
Loft                              35
Single family villa               23
Attic                             14
Building                          12
Single-family detached house       9
Multi-family detached house        5
Multi-family villa                 3
Farmhouse                          2
Country house                      1
Terraced house                     1
Apartment in villa                 1
Name: House_type_final, dtype: int64

In [240]:
# apartment_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Apartment', case=False) == True
# penthouse_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Penthouse', case=False) == True
# loft_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Loft', case=False) == True
# attic_condition = th_no_missing_price['House_type_final'].str.contains(pat=r'Attic', case=False) == True